In [2]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#modelamiento
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

In [3]:
#nativos
import os
import unicodedata

#terceros
import pandas as pd
import pymssql
import missingno as msno
from collections import Counter
import scorecardpy as sc

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#### ACÁ SOLO CAMBIAR EL EXCEL BASE

In [160]:
data = pd.read_excel('BUSQUEDA DE ALERTAS 02_12_2020.xlsx', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,PERIODO,COD_UNICO,NOMBRE_COMPLETO,FLG_VIGENTE,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,FLG_POSICION,...,_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,_50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,_20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL,_int_COVID_ACOGIO_SUSPENSION_PERFECTA,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT,COVID_FICHA_LLENADA
0,202007,0007504210,FABRICA NACIONAL DE ACUMULADORES ETNA,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,False,False,0
1,202007,0001489426,TRANSPORTES ZETRAMSA,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
2,202007,0016343477,CORPORACION OTINGSA,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
3,202007,0014372393,EVANS & TANIA CONSTRUCTORES,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0
4,202007,0014480178,KAESER COMPRESORES DE PERU,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,False,False,0


#### LAS COLUMNAS UQE YA EXITEN

In [161]:
#print(list(data.columns))

In [162]:
for col in [_ for _ in data.columns if 'TIENE' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CUT' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MENOR' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MAYOR' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'SUBIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NUM' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'INT' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CAIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NRO' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'COVID' in _.upper()]:
    data[col] = data[col].fillna(0)

In [163]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} or 'NRO' in col.upper()]
#print(list(cols_combine))
print(len(cols_combine))

778


In [164]:
cols_combine = \
    [col for col in cols_combine if 'SOBREGIRO' not in col.upper() \
                                 and 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and '%' not in col.upper() \
                                 and 'REFINANCI' not in col.upper()
    ]
#print(list(cols_combine))

In [165]:
print(len(cols_combine))

646


In [166]:
for col in ['cambio_seguir_covid', 'TIENE_DIAS_DE_ATRASO_IBK_ESTE_MES', 'TIENE_DEUDA_VIGENTE_IBK', 'IS_NORMAL_IBK_SBS'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

In [167]:
import itertools
combine = list(itertools.combinations(cols_combine, 2))
len(combine)

205120

In [168]:
data['PERIODO'].value_counts()

202007    1924
202008    1882
202009    1832
202010    1801
Name: PERIODO, dtype: int64

#### SOLO TOMAR 202007 Y 202008 ORUQEU SON LOS MESES QUE HASTA EL MONENTO TIENEN TARGET COMPLETO 2 MESES ENADELANTE

In [169]:
banca_a_analizar = 'BEL ZONAL 1'
banca_a_analizar_2 = 'BEL ZONAL 3' # CAMBIAR POR LA ZONA A BUSCAR 
banca_a_analizar_3 = 'BEP ZONAL 1'
banca_a_analizar_4 = 'BEP ZONAL 2'
banca_a_analizar_5 = 'BEP ZONAL 3'
banca_a_analizar_6 = 'BEL ZONAL 2'

In [170]:
data_bel_zon_1= data[
    (data['ZONAL'].isin([banca_a_analizar_6])) & (data['PERIODO'].astype(int).isin([202007, 202008]))
]

In [171]:
assert False

AssertionError: 

#### VALIDACION DE ESTAR TOMANDO LO CORRECTO , DEBE SOLO APRECER LOS PERIODOS DE COSECHA COMPLETA Y LA ZONA INDICADA

In [23]:
data_bel_zon_1['PERIODO'].value_counts()

202007    897
202008    871
Name: PERIODO, dtype: int64

In [24]:
data_bel_zon_1['ZONAL'].value_counts()

BEL ZONAL 2    1768
Name: ZONAL, dtype: int64

## GO!!!!!!!!!!!

In [25]:
[col for col in data_bel_zon_1 if 'target' in col.lower()]

['PRE_TARGET_yoy',
 'target_yoy_quartil_03',
 'target_yoy_quartil_04',
 'target_yoy_quartil_05',
 'target_yoy_quartil_06',
 'target_yoy_quartil_07',
 'target_feve_reducir_salir',
 'target_seguircovid_reducir_salir',
 'cuasi_target_preocupa',
 'target_no_normales_ibk_sbs',
 'target_8_dias_atraso',
 'target_15_dias_atraso',
 'target_rcc_8_dias_atraso',
 'target_SF_8_dias_atraso_o_mas']

In [356]:
data_preocupa = data[
    (
        (data['target_feve_reducir_salir'] == 1) | (data['FEVE'] == 'SEGUIR COVID')
    ) & (data['PERIODO'].astype(int) >= 202009)
]

pd.crosstab(
    data_preocupa['target_feve_reducir_salir'], data_preocupa['PERIODO']
)

PERIODO,202009,202010
target_feve_reducir_salir,,
0.0,0,66
1.0,22,0


In [358]:
clientes_preocupan = data_preocupa[data_preocupa['PERIODO'].astype(int) >= 202009]['COD_UNICO'].unique()
len(clientes_preocupan)

88

In [359]:
_8_dias = data_preocupa = data[
    (data['target_SF_8_dias_atraso_o_mas'] == 1) & (data['PERIODO'].astype(int) >= 202009)
]
pd.crosstab(
    _8_dias['target_SF_8_dias_atraso_o_mas'], _8_dias['PERIODO']
)

PERIODO,202009,202010
target_SF_8_dias_atraso_o_mas,,
1.0,79,43


In [341]:
clientes_8_dias_atraso_2_meses_adlenate = list(_8_dias['COD_UNICO'].unique())
len(clientes_8_dias_atraso_2_meses_adlenate)

43

In [360]:
col_target = 'target_feve_reducir_salir'

#### EN LOS PRINT VERASL LAS VARIABLES LA PROBABILIDA PROMEDIO Y LA CANTIDAD HALLADA PARA IRA ANALIZANDO

In [43]:
import random as rn
listado = []  # acumula lo scaoss ya''q ya validóoooooooo
aucc = {}

In [40]:
from sklearn.metrics import roc_auc_score, recall_score

In [45]:
data_play_2 = data_bel_zon_1[data_bel_zon_1[col_target].notnull()].copy()

In [141]:
set_cols_score_upper_0_3_feve = set(cols_score_upper_0_3_feve)
len(set_cols_score_upper_0_3_feve), len(cols_score_upper_0_3_feve)

(339, 1470)

In [142]:
searach_important = sorted(
    [(col, cols_score_upper_0_3_feve.count(col)) for col in set_cols_score_upper_0_3_feve],
    key=lambda _: _[-1]
)

In [143]:
_150_mas_importantes = [_[0] for _ in  searach_important[-150:] if _[0] in data_bel_zon_1.columns] + ['IS_NORMAL_IBK_SBS', 'target_SF_8_dias_atraso_o_mas', 'COD_UNICO'] 
len(_150_mas_importantes)

149

In [144]:
_150_mas_importantes[:5]

['cut_0_NO_DISMINUYO_TIPOS_CASH_OUT_U3M',
 'F_Septile_cut_nan',
 '_aux_TRX_CAIDA_30PERCENT_DEPOSITOS__MES_ANT',
 '_1_cut_up_NUM_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M',
 '_4_cut_up_F_Septile']

## SIMULACIO    202008   TRATANDO DE ENCONTRARA LOS QUE PREOCUPAN DE LOS PERIODOS 202009 Y 202010

In [145]:
data_simulacion = data_bel_zon_1[data_bel_zon_1['PERIODO'] == '202008'][_150_mas_importantes]
data_simulacion.shape

(871, 149)

In [146]:
len(clientes_preocupan)

191

In [147]:
data_simulacion['TIENE_CLASIFICAION_NO_NORMAL'] = data_simulacion['IS_NORMAL_IBK_SBS'].apply(lambda _: not(_)).astype(int)

In [148]:
pd.crosstab(
    data_simulacion['IS_NORMAL_IBK_SBS'], data_simulacion['TIENE_CLASIFICAION_NO_NORMAL'] 
)

TIENE_CLASIFICAION_NO_NORMAL,0,1
IS_NORMAL_IBK_SBS,,
0,0,44
1,827,0


In [149]:
del  data_simulacion['IS_NORMAL_IBK_SBS'] 

In [151]:
combine_simulacion = list(itertools.combinations(list(data_simulacion.drop(['target_SF_8_dias_atraso_o_mas', 'COD_UNICO'], axis=1).columns), 2))
len(combine_simulacion)

10731

In [152]:
len(clientes_preocupan), col_target

(191, 'target_feve_reducir_salir')

In [154]:
data_play_simulacion = data_simulacion[data_simulacion['target_SF_8_dias_atraso_o_mas'].notnull()].copy()
data_play_simulacion.shape

(515, 149)

In [155]:
dict_1_validation = {}
list_1_valdiacion = []

In [157]:
rn.shuffle(combine_simulacion)
listado =[]

while len(list_1_valdiacion) / len(clientes_preocupan) < 0.95:
    x, y = rn.choice(combine_simulacion)
    col_evaluate = '{}_and_{}'.format(x, y)
    
    if col_evaluate not in listado or 'accionista' not in col_evaluate.lower() or 'gerencia' not in col_evaluate.lower() :
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1
        
        data_simulacion[col_evaluate] = data_simulacion[[x, y]].apply(lambda_basico, axis=1).astype(int)
        data_play_simulacion[col_evaluate] = data_play_simulacion[[x, y]].apply(lambda_basico, axis=1).astype(int)
        
        result = recall_score(data_play_simulacion[col_evaluate], data_play_simulacion['target_SF_8_dias_atraso_o_mas'])
        
        
        if result >= 1.0:
            print("")
            coincidencias = set(data_simulacion[data_simulacion[col_evaluate] == 1]['COD_UNICO'].unique())
            list_1_valdiacion.extend(list(coincidencias & set(clientes_preocupan)))
            dict_1_validation[col_evaluate] = [result, len(coincidencias)]
            

            print(x, y, result, '////', len(coincidencias))
        else:
            print(".", end='')

...
_aux_TRX_SUBIDA_30PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT COVID_ACOGIO_SUSPENSION_PERFECTA_SI 1.0 //// 1
....................................................................
cut_02_INGRESO_SUPERA_EGGRESO_SOLO_EN_20PERCENT_O_MENOS _2_cut_up_NUM_BANCOS_NOIBK_SUBIDA_30PERCENT_MES_ANT 1.0 //// 2
.....................................................................................................
cut_02_INGRESO_SUPERA_EGGRESO_SOLO_EN_20PERCENT_O_MENOS TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC 1.0 //// 1
.............................................................
_aux_TRX_CAIDA_SEGUIDA_DEPOSITOS__U3M F_Septile_cut_6.0 1.0 //// 4
........................................................................................
M_Septile_cut_7.0 TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC 1.0 //// 1
...............................................................................................................................................
TIENE__OTRA_ENTIDAD_PRESTAMOS_S_COMEX_Y_LEASING_VIGENTES_SF_ULT

KeyboardInterrupt: 

In [342]:
_vars = pd.read_excel('Total Alertas - Mediana Empresa.xlsx').sort_values(by=['T_FEVE_PREOCUPA'], ascending=False)
_vars = _vars[_vars['CLIENTES'] > 20]
_vars.head(20)

,Nro,Variable 1,Variable 2,EVALUACION,Unnamed: 4,T_8D_ATRASO,T_FEVE_PREOCUPA,Unnamed: 7,CLIENTES
406,407.0,M_Septile_cut_6.0,_20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,0.344203,////,0.166667,0.521739,====,23
813,NaN,TICKET_PROMEDIO_EN_RIESGO__Bucket 4 - 6.7MM,_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_M...,0.368012,////,0.214286,0.521739,====,23
1381,NaN,cut_letal_SUBSECTOR,_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_M...,0.369176,////,0.222222,0.516129,====,31
1426,NaN,_6_cut_up_M_Septile,cut_letal_SUBSECTOR,0.461302,////,0.409091,0.513514,====,37
1289,NaN,_10_cut_up_CEF_ROTACION_INVENTARIOS,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCC...,0.386364,////,0.272727,0.500000,====,22
1334,NaN,_aux_TRX_SUBIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT,_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,0.295455,////,0.090909,0.500000,====,26
1528,NaN,_5_cut_up_CEF_ROTACION_INVENTARIOS,_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCC...,0.365385,////,0.230769,0.500000,====,24
650,NaN,_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR,_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_M...,0.443902,////,0.400000,0.487805,====,41
1222,NaN,_200_cut_up_CEF_ROTACION_INVENTARIOS,_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,0.341379,////,0.200000,0.482759,====,29
1519,NaN,M_Septile_cut_6.0,_12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL,0.307407,////,0.133333,0.481481,====,27


In [343]:
len(clientes_preocupan), type(clientes_preocupan)

(186, numpy.ndarray)

In [344]:
for row in _vars.sample(frac=1).reset_index(drop=True).iterrows():
    cols_1 = row[1][1]
    cols_2 = row[1][2]
    cantidad = row[1][-1]
    score_feve = row[1][6]
    print(row)
    print(row[1][6])
    break

(0, Nro                                                            53
Variable 1                    _1.5_cut_up_CEF_DTOTAL_S_EBITDA_VAR
Variable 2         _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL
EVALUACION                                               0.276923
Unnamed: 4                                                   ////
T_8D_ATRASO                                              0.153846
T_FEVE_PREOCUPA                                               0.4
Unnamed: 7                                                   ====
CLIENTES                                                       30
Name: 0, dtype: object)
0.4


In [345]:
clientes_a_encontar = set(clientes_8_dias_atraso_2_meses_adlenate) | set(clientes_preocupan)
len(clientes_a_encontar)

217

In [386]:
grupo_alerta = {}
cols_usada = []
lista_coinciden_valida = []
lista_alertados = []
listado_por_alerta = []

for row in _vars.sample(frac=1).reset_index(drop=True).iterrows():
    cols_1 = row[1][1]
    cols_2 = row[1][2]
    cantidad = row[1][-1]
    score_feve = row[1][6]
    
    if score_feve >= 0.3 and cols_1 not in cols_usada and cols_2 not in cols_usada:
        try:
            data_que_cumple = data[
                (data[cols_1] == 1) & (data[cols_2] == 1) & (data['PERIODO'].astype(int) == 202008)
            ]
            cod_unicos_cumplen = set(data_que_cumple['COD_UNICO'])
            listado_por_alerta.append(list(cod_unicos_cumplen))
            cols_usada.append(cols_1)
            cols_usada.append(cols_2)
            
            if len(cod_unicos_cumplen) > 10:
                
                coinciden_feve_duro = cod_unicos_cumplen & clientes_a_encontar
                lista_alertados.extend(cod_unicos_cumplen)
                lista_coinciden_valida.extend(coinciden_feve_duro)
                
                print(cols_1, '|||', cols_2, '||| de: ', len(cod_unicos_cumplen), 'coinciden : ', len(coinciden_feve_duro))
                grupo_alerta[cols_1 + '|'+ cols_2] = 'de: {} coinciden: {}'.format(len(cod_unicos_cumplen), len(coinciden_feve_duro))
            
            if len(set(lista_coinciden_valida)) / len(clientes_a_encontar) > 0.6:
                print("="*100)
                print("De {} alertó bien a: {}".format(
                    len(set(lista_alertados)), len(set(lista_coinciden_valida))
                ))
                
                clientes_detectados_ok = set(lista_coinciden_valida)
                clientes_no_alertar = set(lista_alertados) - set(clientes_detectados_ok)
                
                dicc_cliente_no_alertar = {}
                print("ANALIZANDO CLIENTES ")
                for cliente in clientes_no_alertar:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_no_alertar[cliente] = aparazco
                    
                dicc_cliente_ok = {}
                print("ANALIZANDO OK ")
                for cliente in clientes_detectados_ok:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_ok[cliente] = aparazco
                        
                break
        except Exception as e:
            print(str(e))
            coinciden_feve_duro

_0.8_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR ||| _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL ||| de:  31 coinciden :  12
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR ||| _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL ||| de:  49 coinciden :  22
_3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC ||| _2_cut_up_NUM_BANCOS_DEBT_MENOR_DE_U3M ||| de:  67 coinciden :  18
F_Septile_cut_7.0 ||| _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL ||| de:  50 coinciden :  18
_1.1_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR ||| _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL ||| de:  30 coinciden :  10
cut_letal_SUBSECTOR ||| _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL ||| de:  46 coinciden :  19
cut_0_NINGUN_INGRESO_CASHIN_U3M ||| _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC ||| de:  20 coinciden :  5
NRO_BANCOS_CON_MAYOR_SOW_ULT_RCC ||| _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT ||| de:  13 coinciden :  6
_0.8_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_V

In [387]:
grupo_alerta_1 = grupo_alerta.copy()
len(grupo_alerta_1)

22

In [395]:
max(dicc_cliente_no_alertar.values()), min(dicc_cliente_no_alertar.values()), sum(dicc_cliente_no_alertar.values()) / len(dicc_cliente_no_alertar.values())

(7, 1, 1.743801652892562)

In [396]:
max(dicc_cliente_ok.values()), min(dicc_cliente_ok.values()), sum(dicc_cliente_ok.values()) / len(dicc_cliente_ok.values())

(11, 1, 2.549618320610687)

In [378]:
len(grupo_alerta_2), grupo_alerta_2

(32,
 {'_aux_TRX_CAIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT': 'de: 28 coinciden: 10',
  '_0.6_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR|_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL': 'de: 35 coinciden: 14',
  'DEBT_TOTAL_CAIDA_10PERCENT_MES_ANT|_12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL': 'de: 29 coinciden: 14',
  '_1_cut_up_CEF_DTOTAL_S_EBITDA_VAR|_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 99 coinciden: 30',
  '_1_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC|_3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC': 'de: 72 coinciden: 19',
  '_200_cut_up_CEF_ROTACION_INVENTARIOS|_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 104 coinciden: 37',
  '_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR|_48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 46 coinciden: 19',
  '_1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR|_30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 43 coinciden: 20',
  '_0.2_cu

In [379]:
len(grupo_alerta_1), grupo_alerta_1

(37,
 {'cut_0_NO_DISMINUYO_TIPOS_CASH_OUT_U3M|_50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL': 'de: 41 coinciden: 16',
  '_0.35_cut_down_SOW_IBK_MAX_historia|_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 95 coinciden: 30',
  '_18_cut_up__intFM_category_septil|_3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC': 'de: 26 coinciden: 12',
  'COVID_P_1_TE_AFECTO_COVID_SI|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT': 'de: 59 coinciden: 20',
  '_0.6_cut_down_CEF_CAPITAL_TRABAJO_VAR|_40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 114 coinciden: 41',
  'NRO_BANCOS_NOIBK_SUBIDA_30PERCENT_MES_ANT|_48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 33 coinciden: 8',
  '_0.4_cut_down_CEF_CAPITAL_TRABAJO_VAR|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT': 'de: 34 coinciden: 11',
  'TIENE__OTRA_ENTIDAD_TARJETACREDIT_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC|_0.8_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR': 'de: 24 coincide

In [380]:
len(grupo_alerta_6), grupo_alerta_6

(42,
 {'_1.5_cut_up_CEF_DTOTAL_S_EBITDA_VAR|_int_COVID_ACOGIO_SUSPENSION_PERFECTA': 'de: 36 coinciden: 12',
  'F_Septile_cut_6.0|_48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL': 'de: 34 coinciden: 9',
  '_0.6_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR|_20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL': 'de: 48 coinciden: 20',
  'NO_TIENE_TRX_DEPOSITOS__MES_ACTUAL|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT': 'de: 30 coinciden: 8',
  '_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR|_-10_cut_down_COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019': 'de: 30 coinciden: 11',
  '_6_cut_up_M_Septile|cut_letal_SUBSECTOR': 'de: 25 coinciden: 16',
  '_3_cut_up_NUM_BANCOS_CON_MENOR_SOW_ULT_RCC|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT': 'de: 46 coinciden: 17',
  '_2_cut_up_BUCKET|_int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT': 'de: 30 coinciden: 11',
  '_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC|_30_cut_u

In [309]:
grupo_alerta_8 = grupo_alerta.copy()
len(grupo_alerta_8)

50

In [322]:
grupo_alerta_7 = grupo_alerta.copy()
len(grupo_alerta_7)

61

In [ ]:
assert False # Para que no pase de acá

# SELCCIONADAS PROVINCIA

In [ ]:
........................................................... 
_aux_TRX_CAIDA_20PERCENT_OTROS_NO_TABLERO_MES_ANT IS_NORMAL_IBK_SBS 0.49206349206349204 //// 0_and_0___ 0.6666666666666666 0.31746031746031744  ====  63
....................................................................................................................................................................................................................................................................................................................... 
TIENE_REACTIVA1_REACTIVA2_SF IS_NORMAL_IBK_SBS 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.5 //// 1_and_0___ 0.6666666666666666 0.3333333333333333  ====  12
. 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.4937106918238994 //// 0_and_0___ 0.6666666666666666 0.32075471698113206  ====  53
........................................................................................................................................................................ 
TIENE__OTRA_ENTIDAD_TARJETACREDIT_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC _30_cut_up__intFM_category_septil 0.3611111111111111 //// 1_and_1___ 0.5 0.2222222222222222  ====  9
............................................................................................................................................................ 
TRX_SUBIDA_20PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.5151515151515151 //// 1_and_0___ 0.6666666666666666 0.36363636363636365  ====  11
. 
TRX_SUBIDA_20PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.4907407407407407 //// 0_and_0___ 0.6666666666666666 0.3148148148148148  ====  54
...................................................................................................................................... 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC TIENE_ADEX_PESCA_IMP 0.3333333333333333 //// 0_and_1___ 0.6666666666666666 0.0  ====  6
................................................... 
DEBT_TOTAL_SUBIDA_20PERCENT_MES_ANT _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34650735294117646 //// 0_and_1___ 0.4117647058823529 0.28125  ====  32
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_4_cut_up_R_Quartile _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.5054945054945055 //// 1_and_1___ 0.8571428571428571 0.15384615384615385  ====  13
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_1_cut_down_CEF_ROTACION_CTASXPAGAR_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38888888888888884 //// 1_and_1___ 0.3333333333333333 0.4444444444444444  ====  9
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_0.5_cut_down_CEF_CAPITAL_TRABAJO_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38095238095238093 //// 0_and_1___ 0.42857142857142855 0.3333333333333333  ====  9
................................... 
TRX_CAIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.33055555555555555 //// 0_and_1___ 0.4166666666666667 0.24444444444444444  ====  45
....................................................................................................................................................................................................................................................................... 
_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.6363636363636364 //// 0_and_1___ 1.0 0.2727272727272727  ====  11
.......................................................................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M TICKET_PROMEDIO_EN_RIESGO__Bucket 1 - 159.8 M 0.3333333333333333 //// 1_and_0___ 0.5 0.16666666666666666  ====  6
......... 
IS_NORMAL_IBK_SBS TIENE_ADEX_MADERAS_EXP 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
.................................................................................................. 
ZONAL__BEP ZONAL 3 _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34090909090909094 //// 0_and_1___ 0.5 0.18181818181818182  ====  33
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_1_cut_up_NUM_BANCOS_NOIBK_CAIDA_30PERCENT_MES_ANT _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 1_and_1___ 0.25 0.5  ====  6
................................................................................................................................. 
_aux_TRX_CAIDA_30PERCENT_TRANSFERENCIAS__MES_ANT _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.3388888888888889 //// 0_and_1___ 0.4 0.2777777777777778  ====  18
.................................................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TRX_CAIDA_10PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.65 //// 1_and_0___ 1.0 0.3  ====  10
. 
_aux_TRX_CAIDA_10PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.4818181818181818 //// 0_and_0___ 0.6363636363636364 0.32727272727272727  ====  55
............................................................ 
R_Septile_cut_1.0 _7_cut_up_M_Septile 0.3333333333333333 //// 1_and_1___ 0.6 0.06666666666666667  ====  15
..................................................................................................................................................................................... 
IS_NORMAL_IBK_SBS _0.9_cut_down_COVER_GAR_AUTO_ULT_RCC 0.5168621700879765 //// 0_and_1___ 0.7272727272727273 0.3064516129032258  ====  62
............................................................................... 
IS_NORMAL_IBK_SBS _0.2_cut_down_COVER_GAR_AUTO_ULT_RCC 0.5168621700879765 //// 0_and_1___ 0.7272727272727273 0.3064516129032258  ====  62
................................................................................................... 
_0.8_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_ADEX_PESCA_IMP _0.1_cut_down_COVER_GAR_AUTO_ULT_RCC 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  6
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.3333333333333333 //// 1_and_1___ 0.0 0.6666666666666666  ====  6
............................................................................................................................... 
_aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 1_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
ADEX_CAIDA_40PERCENT_MES_ANT_EXP cat_CASHIN_ACTIVOS_MES_ACTUAL__3 a 4 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
............................................................................................................. 
cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 0_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
......................................................................................................................................................................................... 
_aux_TIENE_TRX_TRANSFERENCIAS__U3M TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 1_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS DEBT_TOTAL_SUBIDA_30PERCENT_MES_ANT 0.4903129657228018 //// 0_and_0___ 0.6363636363636364 0.3442622950819672  ====  61
...................................... 
_0.4_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.39285714285714285 //// 0_and_1___ 0.5 0.2857142857142857  ====  28
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS TIENE_ADEX_AGROPECUARIO_Y_AGROINDUSTRIAS_EXP 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
.......................................................................................................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M DEBT_CAIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.4107142857142857 //// 0_and_1___ 0.7142857142857143 0.10714285714285714  ====  28
............................... 
_aux_TRX_CAIDA_20PERCENT_POS__MES_ANT TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 0_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
............................................................................................................................................................................................................................................. 
TICKET_PROMEDIO_EN_RIESGO__Bucket 2 - 1.0 MM _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.4285714285714286 //// 1_and_1___ 0.5 0.35714285714285715  ====  14
........................................................................................................................................................................................................................... 
COVID_TIENES_PENSADO_O_ESTAS_HACIENDO_INVERSIONES _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
.................... 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.5 //// 1_and_0___ 0.6666666666666666 0.3333333333333333  ====  12
. 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.4937106918238994 //// 0_and_0___ 0.6666666666666666 0.32075471698113206  ====  53
................................................................................................................................... 
_aux_TRX_SUBIDA_30PERCENT_OPERACIONES_MESA__CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.5303030303030303 //// 0_and_0___ 0.7272727272727273 0.3333333333333333  ====  63
..................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_ADEX_PESCA_IMP TIENE_ADEX_VARIOS_EXP 0.40476190476190477 //// 1_and_0___ 0.6666666666666666 0.14285714285714285  ====  7
.................................................................................................................... 
NRO_BANCOS_DEBT_MAYOR_DE_U3M _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.33814102564102566 //// 0_and_1___ 0.38461538461538464 0.2916666666666667  ====  24
............................................................................................................................................... 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.5 //// 0_and_1___ 0.6666666666666666 0.3333333333333333  ====  6
............................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.49206349206349204 //// 0_and_0___ 0.6666666666666666 0.31746031746031744  ====  63
................................................................................................................................................................................................................................................................................................................................................................................................................................ 
_0.7_cut_up_SOW_IBK_ULT_RCC _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3875 //// 1_and_1___ 0.4 0.375  ====  8
................................................................................................................................... 
R_Septile_cut_1.0 _7_cut_up_M_Septile 0.3333333333333333 //// 1_and_1___ 0.6 0.06666666666666667  ====  15
.................................................................................................................................................................................................................................................................................................................................................................................... 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M IS_NORMAL_IBK_SBS 0.4357142857142857 //// 1_and_0___ 0.5714285714285714 0.3  ====  30
. 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M IS_NORMAL_IBK_SBS 0.5714285714285714 //// 0_and_0___ 0.8 0.34285714285714286  ====  35
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_MENOR_OPERACIONES_MESA__DE_U3M IS_NORMAL_IBK_SBS 0.4822443181818182 //// 0_and_0___ 0.6363636363636364 0.328125  ====  64
............................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_OTROS_NO_TABLERO_U3M IS_NORMAL_IBK_SBS 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
...................................................................................................................................... 
ADEX_MENOR_U3M_IMP _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
_aux_TRX_CAIDA_20PERCENT_PAGOS_RECIBIDOS__MES_ANT IS_NORMAL_IBK_SBS 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
.................................................................................................................................................................................... 
_200_cut_up_CEF_ROTACION_INVENTARIOS _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.5416666666666666 //// 1_and_1___ 0.75 0.3333333333333333  ====  21
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3472222222222222 //// 0_and_1___ 0.4444444444444444 0.25  ====  20
....................................................................................................................................................................................................................... 
DEBT_TOTAL_SUBIDA_10PERCENT_MES_ANT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.32887700534759357 //// 0_and_1___ 0.36363636363636365 0.29411764705882354  ====  17
................................................................ 
IS_NORMAL_IBK_SBS DEBT_SUBIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.505464480874317 //// 0_and_0___ 0.6666666666666666 0.3442622950819672  ====  61
..................................................................................................................... 
TIENE_ADEX_PESCA_IMP _1_CAMBIO_GERENCIA_GENERAL_ANIO_ANTERIOR_O_ACTUAL 0.40476190476190477 //// 1_and_0___ 0.6666666666666666 0.14285714285714285  ====  7
.................................................................................................................................................................................................. 
TIENE_EN_OTRAENTIDAD_DEBT_TARJETAS_DE_CREDITO_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.39610389610389607 //// 1_and_1___ 0.42857142857142855 0.36363636363636365  ====  11
............................................................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TRX_CAIDA_20PERCENT_COBRANZAS__MES_ANT TIENE_ADEX_PESCA_IMP 0.41666666666666663 //// 0_and_1___ 0.6666666666666666 0.16666666666666666  ====  6
............................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS TIENE_ADEX_SIDERURGICO_Y_METALURGICO_EXP 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
............................................................ 
_420_cut_up_CEF_ROTACION_INVENTARIOS _0.25_cut_down_SOW_IBK_MAX_historia 0.32539682539682535 //// 1_and_1___ 0.42857142857142855 0.2222222222222222  ====  18
............................................................................................................................................................................................................. 
TIENE_SOLO_2_TIPO_CASH_IN_ACTIVO_ESTE_MES _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34623655913978496 //// 0_and_1___ 0.4666666666666667 0.22580645161290322  ====  31
..................................................................................................................... 
cut_07_EGRESO_SUPERA_INGRESO_EN_30PERCENT_O_MAS _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34090909090909094 //// 1_and_1___ 0.5 0.18181818181818182  ====  22
...................................................................................................................................... 
IS_NORMAL_IBK_SBS _0_cut_down_COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019 0.5 //// 0_and_0___ 0.6666666666666666 0.3333333333333333  ====  63
................................................................................................ 
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _0.85_cut_down_PERCENT_COL_DIRECTAS_SF_MAX_historia 0.6666666666666666 //// 1_and_1___ 1.0 0.3333333333333333  ====  12
......................................................................... 
TIENE_DEUDA_LEASING_VIGENTE_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.36904761904761907 //// 0_and_1___ 0.5 0.23809523809523808  ====  21
........................... 
_1_cut_up_NUM_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.5625 //// 0_and_1___ 1.0 0.125  ====  8
............................................... 
IS_NORMAL_IBK_SBS NRO_BANCOS_NOIBK_CAIDA_30PERCENT_MES_ANT 0.7 //// 0_and_1___ 1.0 0.4  ====  10
 
IS_NORMAL_IBK_SBS NRO_BANCOS_NOIBK_CAIDA_30PERCENT_MES_ANT 0.45454545454545453 //// 0_and_0___ 0.6 0.3090909090909091  ====  55
.......................................................................................................................... 
ZONAL__BEP ZONAL 2 _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  18
........................................................................................................................................................................................................ 
TIENE_TRX_PAGO_DE_SERVICIOS__CASH_OUT_MES_ACTUAL IS_NORMAL_IBK_SBS 0.47391952309985097 //// 0_and_0___ 0.6363636363636364 0.3114754098360656  ====  61
............................................ 
TRX_SUBIDA_SEGUIDA_U3M_CASH_IN ADEX_CAIDA_50PERCENT_MES_ANT_EXP 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  6
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS DEBT_SUBIDA_20PERCENT_MTOSALDO_IBK_SF_U3M 0.505464480874317 //// 0_and_0___ 0.6666666666666666 0.3442622950819672  ====  61
.............................................................................................................. 
_aux_TIENE_TRX_TRANSFERENCIAS__CASH_OUT_U3M _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3402777777777778 //// 0_and_1___ 0.5555555555555556 0.125  ====  16
................................................................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS TIENE_PRESTAMOS_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
DEBT_TOTAL_SUBIDA_50PERCENT_MES_ANT _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3321479374110953 //// 0_and_1___ 0.42105263157894735 0.24324324324324326  ====  37
....................................................................................................................................................................... 
_0.4_cut_down_CEF_CAPITAL_TRABAJO_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3375 //// 0_and_1___ 0.375 0.3  ====  10
............................................................................................................. 
NRO_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.33695652173913043 //// 1_and_1___ 0.5 0.17391304347826086  ====  23
.............................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS TIENE_ADEX_PRENDAS_DE_VESTIR_EXP 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
M_Septile_cut_2.0 _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
................... 
_48_cut_up__intFM_category_septil _-0.1_cut_down_VARIACION_PERCENT_GAR_IBK_U12M 0.5769230769230769 //// 1_and_1___ 1.0 0.15384615384615385  ====  13
............................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS _50_cut_up_CEF_ROTACION_CTASXPAGAR 0.625 //// 0_and_1___ 1.0 0.25  ====  20
 
IS_NORMAL_IBK_SBS _50_cut_up_CEF_ROTACION_CTASXPAGAR 0.47777777777777775 //// 0_and_0___ 0.6 0.35555555555555557  ====  45
..................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_MENOR_TOTAL_DE_U3M_CASH_OUT IS_NORMAL_IBK_SBS 0.4242424242424242 //// 1_and_0___ 0.6666666666666666 0.18181818181818182  ====  11
. 
_aux_TRX_MENOR_TOTAL_DE_U3M_CASH_OUT IS_NORMAL_IBK_SBS 0.5092592592592593 //// 0_and_0___ 0.6666666666666666 0.35185185185185186  ====  54
..................................................................................................................................................................................................................................................................................................................................... 
FLG_POSICION IS_NORMAL_IBK_SBS 0.5276988636363636 //// 1_and_0___ 0.7272727272727273 0.328125  ====  64
........................................................................................................................................................................................................................................................ 
_0.6_cut_down_CEF_CAPITAL_TRABAJO_VAR _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.37727272727272726 //// 0_and_1___ 0.45454545454545453 0.3  ====  20
....... 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
......................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_SUBIDA_30PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.39285714285714285 //// 1_and_1___ 0.5 0.2857142857142857  ====  7
........................................................................... 
DEBT_CAIDA_20PERCENT_MTOSALDO_IBK_SF_U3M COVID_FICHA_LLENADA 0.43333333333333335 //// 1_and_1___ 0.6666666666666666 0.2  ====  10
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_1_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
................................................... 
M_Septile_cut_6.0 COVID_FICHA_LLENADA 0.34375 //// 1_and_1___ 0.5 0.1875  ====  16
........................................................................................................................................... 
COVID_TIENES_PENSADO_O_ESTAS_HACIENDO_INVERSIONES _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
................................................................................................................. 
IS_NORMAL_IBK_SBS _1_cut_up_NUM_BANCOS_NOIBK_CAIDA_20PERCENT_MES_ANT 0.7083333333333334 //// 0_and_1___ 1.0 0.4166666666666667  ====  12
 
IS_NORMAL_IBK_SBS _1_cut_up_NUM_BANCOS_NOIBK_CAIDA_20PERCENT_MES_ANT 0.4509433962264151 //// 0_and_0___ 0.6 0.3018867924528302  ====  53
.................................................................................................................................................................................................. 
TRX_CAIDA_40PERCENT_TOTAL_CASH_OUT_MES_ANT _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34287925696594423 //// 0_and_1___ 0.42105263157894735 0.2647058823529412  ====  34
................................................................................................................................................................................................................................................................................................................................... 
_1.5_cut_up_CEF_DTOTAL_S_VENTAS_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3412698412698413 //// 0_and_1___ 0.5714285714285714 0.1111111111111111  ====  9
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.48484848484848486 //// 0_and_0___ 0.6363636363636364 0.3333333333333333  ====  63
............................................................ 
TIENE_ADEX_PESCA_IMP _3_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.41666666666666663 //// 1_and_1___ 0.6666666666666666 0.16666666666666666  ====  6
..... 
_aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M _5_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.5 0.16666666666666666  ====  6
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3350694444444444 //// 0_and_1___ 0.3888888888888889 0.28125  ====  32
............................................................................................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS _1_cut_up_NUM_BANCOS_NOIBK_SUBIDA_30PERCENT_MES_ANT 0.4821162444113264 //// 0_and_0___ 0.6363636363636364 0.32786885245901637  ====  61
............................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_30PERCENT_TRANSFERENCIAS__MES_ANT IS_NORMAL_IBK_SBS 0.49122807017543857 //// 0_and_0___ 0.6666666666666666 0.3157894736842105  ====  57
................................................................ 
_100_cut_up_CEF_ROTACION_CTASXPAGAR _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.43333333333333335 //// 1_and_1___ 0.6666666666666666 0.2  ====  15
............................................................................................................................................................................................................................................................................................................................................................................................................................. 
_5_cut_up_M_Septile _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
................................................................................................... 
_0.85_cut_down_PERCENT_COL_DIRECTAS_SF_MAX_historia _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34650735294117646 //// 0_and_1___ 0.4117647058823529 0.28125  ====  32
.................................................................................................................................................................................. 
TRX_CAIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.4583333333333333 //// 1_and_0___ 0.6666666666666666 0.25  ====  12
. 
TRX_CAIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.5031446540880503 //// 0_and_0___ 0.6666666666666666 0.33962264150943394  ====  53
 
_150_cut_up_CEF_ROTACION_INVENTARIOS _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.5333333333333333 //// 1_and_1___ 0.8 0.26666666666666666  ====  45
........................................................................................................................................................................................................................................................................................... 
_7_cut_up_R_Septile _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.5625 //// 1_and_1___ 1.0 0.125  ====  8
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_0.7_cut_up_CEF_DTOTAL_S_EBITDA_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0_and_1___ 0.5 0.25  ====  12
........................................................................................................................................................................................... 
R_Septile_cut_4.0 TIENE_ADEX_PESCA_IMP 0.3333333333333333 //// 0_and_1___ 0.5 0.16666666666666666  ====  6
.................. 
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M 0.4388888888888889 //// 1_and_0___ 0.6 0.2777777777777778  ====  18
.................................................................................. 
NRO_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
............................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS ES_ACTIVO_S 0.5276988636363636 //// 0_and_1___ 0.7272727272727273 0.328125  ====  64
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
TRX_SUBIDA_10PERCENT_MES_ANTERIOR _aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M 0.41025641025641024 //// 1_and_0___ 0.6666666666666666 0.15384615384615385  ====  13
................................................................................................................................................................ 
_0.5_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 0_and_1___ 0.5 0.3333333333333333  ====  6
............................. 
NRO_BANCOS_DEBT_SUBIDA_SEGUIDA_DE_U3M _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
................................................... 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.4464285714285714 //// 1_and_1___ 0.75 0.14285714285714285  ====  7
.................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS COVID_ACOGIO_SUSPENSION_PERFECTA_SI 0.4972677595628415 //// 0_and_0___ 0.6666666666666666 0.32786885245901637  ====  61
...................................................................................................... 
IS_NORMAL_IBK_SBS DEBT_CAIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.6666666666666666 //// 0_and_1___ 1.0 0.3333333333333333  ====  15
 
IS_NORMAL_IBK_SBS DEBT_CAIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.4377777777777778 //// 0_and_0___ 0.5555555555555556 0.32  ====  50
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
R_Septile_cut_3.0 TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 0_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
.......................... 
_aux_TRX_CAIDA_SEGUIDA_PAGOS_RECIBIDOS__U3M _3_cut_up_NUM_BANCOS_DEBT_MENOR_DE_U3M 0.5 //// 1_and_0___ 1.0 0.0  ====  7
...................................................................................................................... 
cut_0_NINGUN_INGRESO_CASHIN_U3M TRX_CAIDA_40PERCENT_TOTAL_CASH_OUT_MES_ANT 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
............................................................................................................................. 
IS_NORMAL_IBK_SBS _10_cut_up_CEF_ROTACION_CTASXPAGAR 0.5738095238095238 //// 0_and_1___ 0.8333333333333334 0.3142857142857143  ====  35
 
IS_NORMAL_IBK_SBS _10_cut_up_CEF_ROTACION_CTASXPAGAR 0.41666666666666663 //// 0_and_0___ 0.5 0.3333333333333333  ====  30
.......................................................................................................................... 
_0.7_cut_down_CEF_ROTACION_CTASXPAGAR_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.5 //// 0_and_1___ 0.6666666666666666 0.3333333333333333  ====  6
....................................................... 
_aux_TIENE_TRX_RECAUDACION__U3M TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 0_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
..................................... 
_7_cut_up_F_Septile _-0.2_cut_down_VARIACION_PERCENT_GAR_IBK_U12M 0.38888888888888884 //// 1_and_1___ 0.6666666666666666 0.1111111111111111  ====  9
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_-20_cut_down_COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019 _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.34285714285714286 //// 0_and_1___ 0.2857142857142857 0.4  ====  10
.................................................................................................................................................................................................................................................. 
TIENE_ADEX_PESCA_IMP _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.40476190476190477 //// 1_and_0___ 0.6666666666666666 0.14285714285714285  ====  7
....................................................................................................................................... 
TIENE_ADEX_PESCA_IMP _-0.3_cut_down_VARIACION_PERCENT_GAR_IBK_U12M 0.40476190476190477 //// 1_and_0___ 0.6666666666666666 0.14285714285714285  ====  7
.......................................................................................................................................................................... 
_aux_TRX_SUBIDA_SEGUIDA_OTROS_NO_TABLERO_U3M_CASH_OUT _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 1_and_1___ 0.3333333333333333 0.5  ====  10
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
..................................................................... 
DEBT_CAIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38636363636363635 //// 1_and_1___ 0.5 0.2727272727272727  ====  11
............................. 
_aux_TRX_CAIDA_SEGUIDA_RECAUDACION__U3M TIENE_ADEX_PESCA_IMP 0.40476190476190477 //// 0_and_1___ 0.6666666666666666 0.14285714285714285  ====  7
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_-0.3_cut_down_VARIACION_PERCENT_GAR_IBK_U12M _0.099_cut_up_VARIACION_SOW_OTROS_BANCOS_PRIN_U12M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  6
.................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_OPERACIONES_MESA__U3M TIENE_ADEX_PESCA_IMP 0.41666666666666663 //// 0_and_1___ 0.6666666666666666 0.16666666666666666  ====  6
..................................................... 
FLG_CANCELADO _-10_cut_down_COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019 0.41666666666666663 //// 1_and_1___ 0.6666666666666666 0.16666666666666666  ====  6
.............................................................................................................................................. 
_aux_TRX_CAIDA_10PERCENT_PAGOS_RECIBIDOS__MES_ANT IS_NORMAL_IBK_SBS 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
...................................................................................................................................................................................................................................... 
TIENE_SOLO_2_TIPO_CASH_IN_ACTIVO_ESTE_MES _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34623655913978496 //// 0_and_1___ 0.4666666666666667 0.22580645161290322  ====  31
................................................................................................................................................................................................................................................................................................. 
_aux_TRX_CAIDA_SEGUIDA_OTROS_NO_TABLERO_U3M DEBT_CAIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
................................................................................................................................................................................................................... 
_6_cut_up_F_Septile _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3282828282828283 //// 1_and_1___ 0.5454545454545454 0.1111111111111111  ====  18
....................................................................................................................................................................................................................................................................................... 
TRX_DIFF_CASHIN_CASHOUT_MENOR_DE_U3M _0.999_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.41666666666666663 //// 1_and_1___ 0.5 0.3333333333333333  ====  9
................................................................................................................................................. 
_0.9_cut_up_CEF_DTOTAL_S_EBITDA_VAR _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3507692307692308 //// 0_and_1___ 0.46153846153846156 0.24  ====  25
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
NRO_BANCOS_DEBT_MAYOR_DE_U3M _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4388888888888889 //// 0_and_1___ 0.6 0.2777777777777778  ====  18
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_ADEX_PESCA_IMP _1_cut_up_VARIAICION_NUM_BANCOS_CIERRE_2019 0.41666666666666663 //// 1_and_0___ 0.6666666666666666 0.16666666666666666  ====  6
.............................................................................................................................................................................................................................. 
TRX_SUBIDA_SEGUIDA_U3M_CASH_IN _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.4583333333333333 //// 1_and_1___ 0.25 0.6666666666666666  ====  6
............................................................... 
NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M DEBT_TOTAL_CAIDA_20PERCENT_MES_ANT 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
................................. 
_0.5_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC _2_cut_up_VARIAICION_NUM_BANCOS_CIERRE_2019 0.375 //// 0_and_1___ 0.75 0.0  ====  8
............................................................................................................................................................................................................................. 
_0.8_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.45238095238095233 //// 1_and_1___ 0.5714285714285714 0.3333333333333333  ====  12
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS _3_cut_up_NUM_BANCOS_NOIBK_SUBIDA_10PERCENT_MES_ANT 0.4948717948717949 //// 0_and_0___ 0.6666666666666666 0.3230769230769231  ====  65
...................... 
TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_MES_ACTUAL _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.4571428571428572 //// 0_and_1___ 0.7142857142857143 0.2  ====  15
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
F_Septile_cut_7.0 COVID_P_1_TE_AFECTO_COVID_SI 0.4509803921568627 //// 1_and_1___ 0.6666666666666666 0.23529411764705882  ====  17
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_SUBIDA_SEGUIDA_OTROS_NO_TABLERO_U3M_CASH_OUT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 1_and_1___ 0.25 0.5  ====  6
............................................................................................................... 
_4_cut_up_R_Septile _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.358695652173913 //// 1_and_1___ 0.5 0.21739130434782608  ====  23
..................................................................... 
_5_cut_down_CEF_ROTACION_CTASXPAGAR_VAR _2_cut_up_VARIAICION_NUM_BANCOS_CON_MINIMO_U12M 0.35 //// 0_and_1___ 0.5 0.2  ====  10
............................. 
_20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3431372549019608 //// 1_and_1___ 0.3333333333333333 0.35294117647058826  ====  17
.................................................................... 
TIENE_REACTIVA_SF IS_NORMAL_IBK_SBS 0.3269230769230769 //// 1_and_0___ 0.5 0.15384615384615385  ====  13
. 
TIENE_REACTIVA_SF IS_NORMAL_IBK_SBS 0.5326923076923077 //// 0_and_0___ 0.7 0.36538461538461536  ====  52
................................................................................................................................................................................................ 
_aux_TRX_SUBIDA_10PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT _5_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 1_and_1___ 0.3333333333333333 0.5  ====  6
............................................................................................................................................................... 
_1_cut_up_NUM_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.33695652173913043 //// 1_and_1___ 0.5 0.17391304347826086  ====  23
............................................................................................... 
_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.34134615384615385 //// 1_and_1___ 0.375 0.3076923076923077  ====  13
................................................................................................................................................................................................................................................................................................. 
_aux_TIENE_TRX_DEPOSITOS__U3M _-10_cut_down_COVID_P_5_VARIACION_PORCENTUAL_VENTAS_2020_2019 0.43846153846153846 //// 0_and_1___ 0.8 0.07692307692307693  ====  13
................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# SELCCIONADAS GRAN EMPRESA

In [ ]:
........................................................................................................................................................ 
_aux_TIENE_TRX_POS__U3M _1_cut_up_NUM_BANCOS_DEBT_SUBIDA_SEGUIDA_DE_U3M 0.35416666666666663 //// 1_and_1___ 0.375 0.3333333333333333  ====  12
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_0.999_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _7_cut_up_M_Septile 0.35416666666666663 //// 1_and_1___ 0.3333333333333333 0.375  ====  8
............................................................................................................................................................................................................................................................................................... 
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _2_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.45238095238095233 //// 1_and_1___ 0.5714285714285714 0.3333333333333333  ====  12
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_OUT_U3M ADEX_MAYOR_U3M_EXP 0.5 //// 1_and_1___ 1.0 0.0  ====  6
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _1_cut_up_cut_0_TIENE_EL_MISMO_NUM_TIPOS_CAHIN_EN_U3M 0.3571428571428571 //// 0_and_1___ 0.14285714285714285 0.5714285714285714  ====  7
........................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_OPERACIONES_MESA__U3M_CASH_OUT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_0.8_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _3_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M 0.3888888888888889 //// 1_and_1___ 0.5 0.2777777777777778  ====  18
........ 
BACKLOG__Proyectos _7_cut_up_R_Septile 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
.............................................................................................................. 
TIENE_DEUDA_COMEX_VIGENTE_RCC BACKLOG__Proyectos 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
..................................................... 
ADEX_CAIDA_30PERCENT_MES_ANT_EXP cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
.................................................................................................................................................................. 
NO_TIENE_TRX_DEPOSITOS__MES_ACTUAL NO_TIENE_TRX_POS__MES_ACTUAL 0.3492063492063492 //// 1_and_0___ 0.5555555555555556 0.14285714285714285  ====  14
.............................................................. 
M_Septile_cut_7.0 _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.675 //// 1_and_1___ 0.6 0.75  ====  8
................................................... 
NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3611111111111111 //// 1_and_1___ 0.5 0.2222222222222222  ====  9
................................................. 
TRX_CAIDA_50PERCENT_MES_ANTERIOR BACKLOG__Proyectos 0.3875 //// 0_and_1___ 0.4 0.375  ====  8
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
NRO_BANCOS_NOIBK_CAIDA_10PERCENT_MES_ANT _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.34285714285714286 //// 0_and_1___ 0.4 0.2857142857142857  ====  7
.................................................................................................. 
M_Septile_cut_7.0 NRO_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M 0.32575757575757575 //// 1_and_0___ 0.3333333333333333 0.3181818181818182  ====  22
........................................................................................................................................ 
_0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC cut_letal_SUBSECTOR 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  6
...................................................................................................................................................................................... 
BACKLOG__Proyectos ZONAL__BEP ZONAL 3 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
........................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _205_cut_up_CEF_ROTACION_CTASXPAGAR 0.5 //// 1_and_0___ 1.0 0.0  ====  7
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_1_cut_down_CEF_ROTACION_CTASXPAGAR_VAR cut_letal_SUBSECTOR 0.41666666666666663 //// 1_and_1___ 0.3333333333333333 0.5  ====  8
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_2_cut_up_NUM_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  11
.................................................................. 
R_Septile_cut_4.0 BACKLOG__Proyectos 0.39285714285714285 //// 0_and_1___ 0.5 0.2857142857142857  ====  7
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3666666666666667 //// 0_and_1___ 0.4 0.3333333333333333  ====  6
....................... 
TIENE_EN_OTRAENTIDAD_DEBT_TARJETAS_DE_CREDITO_VIGENTE_SF_ULT_RCC cut_letal_SUBSECTOR 0.41428571428571426 //// 0_and_1___ 0.4 0.42857142857142855  ====  7
....................................................................................................... 
_0.7_cut_up_SOW_IBK_ULT_RCC cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
....................................................... 
_aux_TRX_SUBIDA_10PERCENT_OTROS_NO_TABLERO_CASH_OUT_MES_ANT TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M 0.5 //// 0_and_1___ 1.0 0.0  ====  7
................................................................................................................................................................................................................................................. 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _0.7_cut_up_SOW_IBK_ULT_RCC 0.4642857142857143 //// 1_and_1___ 0.5 0.42857142857142855  ====  14
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC 0.5 //// 1_and_1___ 1.0 0.0  ====  6
..................................................................................................................................................................................................... 
M_Septile_cut_5.0 cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
....................................................................................................................... 
_aux_TRX_MENOR_DEPOSITOS__DE_U3M cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
. 
TIENE_TRX_PAGO_DE_SERVICIOS__CASH_OUT_MES_ACTUAL TIENE_ADEX_TEXTIL_EXP 0.39285714285714285 //// 1_and_1___ 0.5 0.2857142857142857  ====  7
................................................................................................................................................................................................... 
_aux_TIENE_TRX_PAGOS_MASIVOS__CASH_OUT_U3M _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.39285714285714285 //// 1_and_1___ 0.5 0.2857142857142857  ====  7
.................................................................................................................................................................................................................................................................................................................................................................... 
BACKLOG__Proyectos _2_cut_up_NUM_VECES_REACTIVA_SF 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
...................................... 
NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.37948717948717947 //// 1_and_1___ 0.4 0.358974358974359  ====  39
..................................................................................................... 
_2_cut_up_NUM_BANCOS_NOIBK_CAIDA_30PERCENT_MES_ANT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
... 
FLG_VIGENTE TIENE_ADEX_MADERAS_IMP 0.41666666666666663 //// 0_and_1___ 0.5 0.3333333333333333  ====  6
............................................................................. 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _18_cut_up__intFM_category_septil 0.3545751633986928 //// 1_and_1___ 0.2777777777777778 0.43137254901960786  ====  51
.................................................................................................................................................................................................. 
_1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  10
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
_4_cut_up_BUCKET _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3666666666666667 //// 1_and_1___ 0.4 0.3333333333333333  ====  9
................................................................................................................................................................................................................................................................................................... 
_6_cut_up_F_Septile _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC 0.38095238095238093 //// 1_and_1___ 0.42857142857142855 0.3333333333333333  ====  15
............................................................................................................................................................................ 
BACKLOG__Proyectos _1_cut_up_CANTIDAD_TRX_CASH_OUT_PERCENT 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
.................................................................................................................. 
FLG_SUSPENDIDO TIENE_ADEX_PRENDAS_DE_VESTIR_EXP 0.375 //// 1_and_1___ 0.5 0.25  ====  8
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS _0_cut_down_VARIACION_PERCENT_GAR_IBK_U12M 0.3657407407407407 //// 0_and_0___ 0.25 0.48148148148148145  ====  27
................................................................................................................................................................................................................................................................................................................................................................................................................ 
F_Septile_cut_3.0 _4_cut_up_NUM_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  10
............ 
NRO_BANCOS_PRIN_DEUDASF_ULT_RCC _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  11
............................................................................................................................................ 
_1_CAMBIO_GERENCIA_GENERAL_ANIO_ACTUAL _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
............................................................................................................... 
_aux_TRX_CAIDA_20PERCENT_DEPOSITOS__MES_ANT _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 0_and_1___ 0.5 0.16666666666666666  ====  6
........................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_PAGOS_RECIBIDOS__U3M TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M 0.5 //// 0_and_1___ 1.0 0.0  ====  7
............................................................................................................................................................................................................... 
TIENE_TRX_OPERACIONES_MESA__CASH_OUT_MES_ACTUAL cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
........................................................................................................................................... 
TIENE_ADEX_PETROLEO_Y_GAS_NATURAL_IMP BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
.......................................................................................................................................................................................... 
NO_TIENE_TRX_RECAUDACION__MES_ACTUAL _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  11
.................................................................................................................... 
_aux_TRX_MENOR_COBRANZAS__DE_U3M cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
........................................................................................................................................................................................................................ 
F_Septile_cut_5.0 _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  10
.............................................................................................................................................................................................................................................. 
cut_01_INGRESO_SUPERA_EGGRESO_SOLO_EN_10PERCENT_O_MENOS _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
.................. 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _0.3_cut_down_SOW_IBK_MIN_historia 0.3542857142857143 //// 1_and_0___ 0.42857142857142855 0.28  ====  25
.............................. 
TIENE_REACTIVA_IBK _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
................................................................................................................................................................................................................................................................................................................................ 
_aux_TRX_CAIDA_SEGUIDA_PAGOS_MASIVOS__U3M_CASH_OUT _2_cut_up_NUM_BANCOS_PRIN_DEUDASF_ULT_RCC 0.5357142857142857 //// 1_and_0___ 0.5 0.5714285714285714  ====  7
....................................................................................................................................................................................................................................................................................................................................... 
BACKLOG__Proyectos _2_cut_up_NUM_BANCOS_NOIBK_SUBIDA_10PERCENT_MES_ANT 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
...................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE__OTRA_ENTIDAD_TARJETACREDIT__Y_LEASING_VIGENTES_SF_ULT_RCC TIENE_ADEX_TEXTIL_EXP 0.33333333333333337 //// 1_and_1___ 0.25 0.4166666666666667  ====  12
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
cut_0_NINGUN_INGRESO_CASHIN_U3M cut_07_EGRESO_SUPERA_INGRESO_EN_30PERCENT_O_MAS 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  6
................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _40_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.5 //// 1_and_0___ 1.0 0.0  ====  7
.................................................................................................................................................................................................................................................. 
_0.3_cut_down_CEF_CAPITAL_TRABAJO_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41428571428571426 //// 1_and_1___ 0.4 0.42857142857142855  ====  7
....................................................................................................................................................................................................................................................................................................................................................................................................................... 
TIENE__OTRA_ENTIDAD_PRESTAMOS_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 1_and_1___ 0.5 0.3333333333333333  ====  6
........................................................................................................................................................................................................................................................................................................................................................................................................................ 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M 0.34444444444444444 //// 1_and_0___ 0.26666666666666666 0.4222222222222222  ====  45
................................................................................................................................................................................................................................................................... 
_50_cut_up_CEF_ROTACION_CTASXPAGAR _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  7
............................................................................................................................................ 
_0.6_cut_up_SOW_IBK_ULT_RCC cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
............................................................................................................................................................... 
cut_06_EGRESO_SUPERA_INGRESO_EN_40PERCENT_O_MAS cut_letal_SUBSECTOR 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  6
................................................................................... 
_2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3666666666666667 //// 0_and_1___ 0.4 0.3333333333333333  ====  9
............................................... 
_aux_TRX_SUBIDA_30PERCENT_PAGO_DE_SERVICIOS__CASH_OUT_MES_ANT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35416666666666663 //// 0_and_1___ 0.3333333333333333 0.375  ====  8
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_30PERCENT_COBRANZAS__MES_ANT BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
.................. 
_aux_TRX_MENOR_PAGO_DE_SERVICIOS__DE_U3M_CASH_OUT NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC 0.38095238095238093 //// 1_and_0___ 0.3333333333333333 0.42857142857142855  ====  7
.............. 
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _7_cut_up_M_Septile 0.4365079365079365 //// 1_and_1___ 0.42857142857142855 0.4444444444444444  ====  9
............................................................................................................. 
_4_cut_up_R_Septile _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0_and_1___ 0.5 0.25  ====  8
................................................................................................................................................................................................................................................................... 
NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC _5_cut_up_F_Septile 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  12
................. 
_0.8_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.3514705882352941 //// 1_and_1___ 0.35 0.35294117647058826  ====  34
....................................................................................................................................................................................................... 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3392857142857143 //// 1_and_1___ 0.25 0.42857142857142855  ====  7
....................... 
_aux_TRX_CAIDA_30PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.39230769230769236 //// 1_and_1___ 0.4 0.38461538461538464  ====  13
....................................................................................................................................................................................................................................................................................................... 
M_Septile_cut_5.0 _3_cut_up_CEF_DTOTAL_S_EBITDA_VAR 0.4772727272727273 //// 1_and_1___ 0.5 0.45454545454545453  ====  11
..................................................................................................................................................................................................................................................................................................................................................... 
NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3333333333333333 //// 0_and_1___ 0.2222222222222222 0.4444444444444444  ====  27
........................................................................................... 
F_Septile_cut_3.0 BACKLOG__Proyectos 0.35416666666666663 //// 0_and_1___ 0.3333333333333333 0.375  ====  8
........................................................................................................................................................................................................................................ 
_aux_TIENE_TRX_OPERACIONES_MESA__CASH_OUT_U3M IS_NORMAL_IBK_SBS 0.3472222222222222 //// 0_and_0___ 0.25 0.4444444444444444  ====  36
........................................................................................................................................................................................................................................................................................ 
_18_cut_up__intFM_category_septil _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4 //// 1_and_1___ 0.3333333333333333 0.4666666666666667  ====  30
........................................................................................................................... 
_4_cut_up_NUM_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC _1_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M 0.35714285714285715 //// 1_and_1___ 0.7142857142857143 0.0  ====  10
............................................................................................................................................................................................................................................................................................. 
M_Septile_cut_1.0 _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3357592446892211 //// 0_and_1___ 0.2682926829268293 0.4032258064516129  ====  62
.............................................................. 
_aux_TRX_CAIDA_SEGUIDA_PAGOS_MASIVOS__U3M_CASH_OUT _1_cut_up_NUM_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M 0.41666666666666663 //// 1_and_0___ 0.3333333333333333 0.5  ====  10
............................... 
BACKLOG__Proyectos _6_cut_up_M_Septile 0.3333333333333333 //// 1_and_0___ 0.5 0.16666666666666666  ====  6
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.37948717948717947 //// 1_and_1___ 0.4 0.358974358974359  ====  39
...................................................................................................................................................................................................................................................................... 
TIENE_ADEX_EXP _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.375 //// 0_and_0___ 0.75 0.0  ====  9
................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M _0.005_cut_down_PERCENT_GAR_HIPO 0.375 //// 0_and_0___ 0.75 0.0  ====  8
................................................................................................................................................................. 
_0.3_cut_down_SOW_IBK_MIN_historia _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  11
....................................................................................................................................................... 
_0.05_cut_down_COVER_GAR_AUTO_ULT_RCC _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  11
.............................................................................................................................................................................. 
_aux_TRX_MENOR_PAGO_DE_SERVICIOS__DE_U3M_CASH_OUT DEBT_SUBIDA_20PERCENT_MTOSALDO_IBK_SF_U3M 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
..... 
_0.15_cut_down_SOW_IBK_MIN_historia cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
.................................................................................................................................................................................................. 
VARIAICION_NRO_BANCOS_PRIN_DEBT_u12m _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.375 //// 1_and_0___ 0.75 0.0  ====  6
........................................................................................................................................................................................................................................................................................................................................................................................................ 
_aux_TRX_CAIDA_10PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT cut_letal_SUBSECTOR 0.43333333333333335 //// 0_and_1___ 0.2 0.6666666666666666  ====  6
........................................................................................................................................................................................................................................................................................................................................................................ 
_aux_TRX_CAIDA_30PERCENT_RECAUDACION__MES_ANT _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  11
............................................................................................. 
cut_0_NINGUN_INGRESO_CASHIN_U3M _1_cut_up_CANTIDAD_TRX_CASH_OUT_PERCENT 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  6
................................................................................................................................................................................................................................................ 
NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC _0.7_cut_down_COVER_GAR_AUTO_ULT_RCC 0.375 //// 0_and_1___ 0.0 0.75  ====  8
................................................................................................... 
_aux_TRX_MAYOR_OTROS_NO_TABLERO_DE_U3M_CASH_OUT _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  6
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M cut_letal_SUBSECTOR 0.43333333333333335 //// 0_and_1___ 0.2 0.6666666666666666  ====  6
......................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _0.4_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC 0.5 //// 1_and_1___ 1.0 0.0  ====  7
........................................................................................................................................................... 
DEBT_SUBIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF _1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.6442307692307692 //// 1_and_1___ 0.75 0.5384615384615384  ====  13
................................................................................................................................................................................................. 
_0.1_cut_down_SOW_IBK_ULT_RCC cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
cat_CASHIN_ACTIVOS_MES_ACTUAL__3 a 4 cut_letal_SUBSECTOR 0.5 //// 0_and_1___ 0.3333333333333333 0.6666666666666666  ====  6
.................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_DEPOSITOS__U3M _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.4 //// 0_and_0___ 0.8 0.0  ====  10
...................... 
TIENE_ADEX_PRENDAS_DE_VESTIR_EXP cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
....................................................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_PAGO_DE_SERVICIOS__U3M_CASH_OUT TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M 0.5 //// 0_and_1___ 1.0 0.0  ====  7
........ 
R_Septile_cut_1.0 IS_NORMAL_IBK_SBS 0.3956521739130435 //// 0_and_0___ 0.4 0.391304347826087  ====  23
.................. 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _30_cut_up__intFM_category_septil 0.3571428571428571 //// 0_and_1___ 0.14285714285714285 0.5714285714285714  ====  7
.................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_SUBIDA_20PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.3345665961945032 //// 0_and_0___ 0.22727272727272727 0.4418604651162791  ====  43
...................................................................................................................................................................... 
NRO_BANCOS_NOIBK_CAIDA_10PERCENT_MES_ANT cut_letal_SUBSECTOR 0.41428571428571426 //// 0_and_1___ 0.4 0.42857142857142855  ====  7
............................................................................ 
DEBT_NO_MTOSALDO_IBK_SF_U3M _1_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC 0.35714285714285715 //// 0_and_0___ 0.0 0.7142857142857143  ====  7
.............................................................................................. 
NRO_BANCOS_CON_MAYOR_SOW_ULT_RCC _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.32575757575757575 //// 0_and_1___ 0.3181818181818182 0.3333333333333333  ====  39
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
M_Septile_cut_nan BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
............................................................... 
NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC _5_cut_up_R_Septile 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  9
.......................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M DEBT_CAIDA_30PERCENT_MTOSALDO_IBK_SF_U3M 0.5 //// 1_and_0___ 1.0 0.0  ====  6
....................................................................................................... 
M_Septile_cut_7.0 _5_cut_up_R_Septile 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  9
................................................................ 
NRO_VECES_REACTIVA_SF _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
............................................................................................................................................................................................................................................................................................................... 
_1.1_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR cut_letal_SUBSECTOR 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  6
................................... 
_aux_TRX_CAIDA_SEGUIDA_COBRANZAS__U3M BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
................................................................................... 
_3_cut_up_NUM_VECES_REACTIVA_SF cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
................................................................................................................................................................................................................................................................ 
_2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M _3_cut_up_NUM_BANCOS_NOIBK_SUBIDA_30PERCENT_MES_ANT 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  11
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
TIENE_ADEX_QUIMICO_EXP _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
....................................................................................................................................... 
TIENE_DEUDA_VIGENTE_RCC _0.3_cut_down_COVER_GAR_AUTO_ULT_RCC 0.375 //// 0_and_1___ 0.0 0.75  ====  8
............................................... 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M 0.5 //// 0_and_1___ 1.0 0.0  ====  7
..................................................................................................................................................................................................................................................................... 
_aux_TRX_CAIDA_30PERCENT_OTROS_NO_TABLERO_CASH_OUT_MES_ANT _2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.38095238095238093 //// 1_and_1___ 0.42857142857142855 0.3333333333333333  ====  15
............................................................................................................................. 
_aux_TRX_CAIDA_SEGUIDA_DEPOSITOS__U3M _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
............................................................................................. 
_1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.37948717948717947 //// 1_and_1___ 0.4 0.358974358974359  ====  39
............. 
TIENE_ADEX_MINERIA_TRADICIONAL_EXP BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
...................................................... 
_150_cut_up_CEF_ROTACION_INVENTARIOS _1_cut_up_NUM_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC 0.41558441558441556 //// 1_and_0___ 0.2857142857142857 0.5454545454545454  ====  11
................................... 
M_Septile_cut_4.0 _0.1_cut_down_SOW_IBK_MIN_historia 0.3380952380952381 //// 1_and_0___ 0.3333333333333333 0.34285714285714286  ====  35
.......................................................................................................................................................................................................... 
_2_cut_up_NUM_VECES_REACTIVA_IBK _3_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  7
........................................................................................................................................................................................................................................................................................................................... 
_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC cut_deCuidado_SUBSECTOR 0.4358974358974359 //// 1_and_1___ 0.3333333333333333 0.5384615384615384  ====  13
......... 
TRX_CAIDA_40PERCENT_TOTAL_CASH_OUT_MES_ANT _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3666666666666667 //// 0_and_1___ 0.4 0.3333333333333333  ====  9
..................... 
_2_cut_up_NUM_BANCOS_DEBT_SUBIDA_SEGUIDA_DE_U3M cut_deCuidado_SUBSECTOR 0.375 //// 1_and_1___ 0.5 0.25  ====  8
............................................................................................................................................................................................................................................. 
NRO_BANCOS_NOIBK_SUBIDA_10PERCENT_MES_ANT _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3333333333333333 //// 0_and_1___ 0.6666666666666666 0.0  ====  6
................................................................................................. 
_aux_TRX_CAIDA_SEGUIDA_OTROS_NO_TABLERO_U3M_CASH_OUT _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3666666666666667 //// 1_and_1___ 0.3333333333333333 0.4  ====  10
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
BACKLOG__Proyectos _0.1_cut_down_SOW_IBK_MAX_historia 0.38095238095238093 //// 1_and_0___ 0.3333333333333333 0.42857142857142855  ====  7
.. 
_aux_TRX_CAIDA_30PERCENT_OTROS_NO_TABLERO_CASH_OUT_MES_ANT _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4888888888888889 //// 1_and_1___ 0.4444444444444444 0.5333333333333333  ====  15
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.5227272727272727 //// 1_and_1___ 0.5454545454545454 0.5  ====  22
....................................................................................................................................................... 
BACKLOG__Proyectos _0.3_cut_down_CEF_CAPITAL_TRABAJO_VAR 0.41666666666666663 //// 1_and_1___ 0.3333333333333333 0.5  ====  6
............................................................... 
TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VIGENTE_SF_ULT_RCC _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.75 //// 1_and_1___ 1.0 0.5  ====  6
............................................ 
_aux_TRX_CAIDA_SEGUIDA_PAGOS_MASIVOS__U3M_CASH_OUT _0.2_cut_down_SOW_IBK_MIN_historia 0.39583333333333337 //// 1_and_0___ 0.375 0.4166666666666667  ====  12
................................................................................................................................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.5 //// 1_and_0___ 1.0 0.0  ====  7
....................................................................................................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M TIENE_ADEX_PETROLEO_Y_GAS_NATURAL_IMP 0.5 //// 1_and_0___ 1.0 0.0  ====  7
........................................................................................................................................................ 
_aux_TRX_MENOR_OTROS_NO_TABLERO_DE_U3M BACKLOG__Proyectos 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M M_Septile_cut_7.0 0.3571428571428571 //// 0_and_1___ 0.14285714285714285 0.5714285714285714  ====  7
................................................................................................... 
NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC _18_cut_up__intFM_category_septil 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  12
.................................................................................. 
_aux_TRX_SUBIDA_10PERCENT_TOTAL_CASH_OUT_MES_ANT IS_NORMAL_IBK_SBS 0.33088235294117646 //// 1_and_0___ 0.25 0.4117647058823529  ====  17
................................................................................................................................................... 
BACKLOG__Proyectos _0.9_cut_up_CEF_DTOTAL_S_VENTAS_VAR 0.39285714285714285 //// 1_and_0___ 0.5 0.2857142857142857  ====  7
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
TIENE_DEUDA_VIGENTE_RCC _16_cut_up__int_FM_category_quartil 0.3333333333333333 //// 0_and_1___ 0.0 0.6666666666666666  ====  9
........................................................................................................................................................................................................................................................................... 
_16_cut_up__int_FM_category_quartil _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.35 //// 0_and_1___ 0.4 0.3  ====  10
...... 
cut_0_TIENE_EL_MISMO_NRO_TIPOS_CAHIN_EN_U3M _aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M 0.375 //// 1_and_0___ 0.25 0.5  ====  8
.......... 
_aux_TRX_MENOR_TRANSFERENCIAS__DE_U3M _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.5 //// 1_and_1___ 0.5 0.5  ====  6
........................................................................... 
_1.1_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.39428571428571424 //// 1_and_1___ 0.42857142857142855 0.36  ====  25
........................................................................................................................................... 
TIENE_ADEX_TEXTIL_IMP _18_cut_up__intFM_category_septil 0.32727272727272727 //// 1_and_1___ 0.2 0.45454545454545453  ====  11
....................................................... 
_6_cut_up_M_Septile _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.45141065830721006 //// 1_and_1___ 0.45454545454545453 0.4482758620689655  ====  29
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TIENE_TRX_PAGOS_RECIBIDOS__U3M cut_letal_SUBSECTOR 0.45238095238095233 //// 0_and_1___ 0.3333333333333333 0.5714285714285714  ====  7
....................................................................................................................................................................................................... 
TIENE_1_O_MAS_BANCOS_CON_MAYOR_SOW_ULT_RCC _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.32575757575757575 //// 0_and_1___ 0.3181818181818182 0.3333333333333333  ====  39
....................................................................................... 
_0.15_cut_down_SOW_IBK_ULT_RCC cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
. 
BACKLOG__Proyectos _1_cut_up_cut_0_TIENE_EL_MISMO_NUM_TIPOS_CAHIN_EN_U3M 0.375 //// 1_and_1___ 0.25 0.5  ====  6
..................................................................................................................................................................................................................................... 
_1_cut_up_NUM_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.39285714285714285 //// 0_and_1___ 0.5 0.2857142857142857  ====  7
.................................................................................................................................................................................. 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _3_cut_up_VARIAICION_NUM_BANCOS_PRIN_DEBT_u12m 0.5 //// 1_and_0___ 1.0 0.0  ====  7
........................................................................................................................... 
_5_cut_up_F_Septile _1_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC 0.3333333333333333 //// 1_and_0___ 0.0 0.6666666666666666  ====  12
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
_2_cut_up_NUM_BANCOS_NOIBK_SUBIDA_10PERCENT_MES_ANT cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
NO_TIENE_TRX_POS__MES_ACTUAL cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M 0.41666666666666663 //// 0_and_1___ 0.5 0.3333333333333333  ====  6
.......................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M ZONAL__BEP ZONAL 3 0.5 //// 1_and_0___ 1.0 0.0  ====  7
.............................. 
TRX_SUBIDA_50PERCENT_MES_ANTERIOR TIENE_ADEX_TEXTIL_EXP 0.38095238095238093 //// 1_and_1___ 0.3333333333333333 0.42857142857142855  ====  7
................................................................................................................................................................................................................................................................................................................................................................ 
_1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _0.2_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC 0.37621621621621626 //// 1_and_0___ 0.32 0.43243243243243246  ====  37
........................................................................................................................................................................................................................ 
_0.2_cut_down_PERCENT_GAR_HIPO _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0_and_1___ 0.5 0.25  ====  8
...................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _0.6_cut_up_SOW_IBK_ULT_RCC 0.5 //// 1_and_0___ 1.0 0.0  ====  6
.............................................................................................................. 
_0.5_cut_up_SOW_IBK_ULT_RCC cut_deCuidado_SUBSECTOR 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  12
...................................................... 
FLG_MONOLINEA _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 0_and_0___ 0.6666666666666666 0.0  ====  11
............................................................................................. 
M_Septile_cut_4.0 _0.1_cut_down_SOW_IBK_MIN_historia 0.3380952380952381 //// 1_and_0___ 0.3333333333333333 0.34285714285714286  ====  35
.......................................................................................................................................... 
_30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL COVID_FICHA_LLENADA 0.3666666666666667 //// 1_and_1___ 0.4 0.3333333333333333  ====  9
..................... 
TIENE_PRESTAMOS_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC cut_letal_SUBSECTOR 0.41666666666666663 //// 0_and_1___ 0.3333333333333333 0.5  ====  8
....................................................................................................................................................................................................... 
_aux_TRX_CAIDA_20PERCENT_COBRANZAS__MES_ANT TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M 0.5 //// 0_and_1___ 1.0 0.0  ====  7
............................................................................................................................................................................................................................................................................................................................................................................................................................................................ 
IS_NORMAL_IBK_SBS TIENE_DEUDA_VIGENTE_RCC 0.375 //// 0_and_0___ 0.0 0.75  ====  8
...... 
TIENE__OTRA_ENTIDAD_PRESTAMOS_Y__TARJETACREDIT_VIGENTES_SF_ULT_RCC cut_letal_SUBSECTOR 0.41428571428571426 //// 0_and_1___ 0.4 0.42857142857142855  ====  7
................................................. 
TRX_CAIDA_SEGUIDA_U3M_CASH_IN _0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.34285714285714286 //// 1_and_1___ 0.2857142857142857 0.4  ====  10
....................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
_aux_TRX_MENOR_OTROS_NO_TABLERO_DE_U3M_CASH_OUT _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.35119047619047616 //// 1_and_1___ 0.2857142857142857 0.4166666666666667  ====  12
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M _7_cut_up_M_Septile 0.32575757575757575 //// 0_and_1___ 0.3333333333333333 0.3181818181818182  ====  22
....................................................... 
_aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M _30_cut_up__intFM_category_septil 0.3571428571428571 //// 0_and_1___ 0.14285714285714285 0.5714285714285714  ====  7
......................................................................................... 
BACKLOG__Proyectos _0.2_cut_down_COVER_GAR_AUTO_ULT_RCC 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  9
............................................................... 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _1.1_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.39428571428571424 //// 1_and_1___ 0.42857142857142855 0.36  ====  25
......................................................................................................................................................................................................................................................... 
TIENE_DEUDA_FACTORING_VIGENTE_RCC BACKLOG__Proyectos 0.35416666666666663 //// 0_and_1___ 0.3333333333333333 0.375  ====  8
.................................................................................................................................................. 
BACKLOG__Proyectos _3_cut_up_NUM_BANCOS_DEBT_CAIDA_SEGUIDA_DE_U3M 0.41666666666666663 //// 1_and_0___ 0.5 0.3333333333333333  ====  6
....................................................................................................................................................................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M _3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.5 //// 1_and_0___ 1.0 0.0  ====  7
................................................................................................... 
TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_OUT_U3M ADEX_CAIDA_50PERCENT_MES_ANT_IMP 0.5 //// 1_and_0___ 1.0 0.0  ====  6
... 
TIENE_DEUDA_PRESTAMOS_VIGENTE_RCC _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  11
.............................................

# SELCCIONADAS MEDIANAS

In [ ]:
............................................................................................ 
_0.3_cut_down_CEF_CAPITAL_TRABAJO_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.36136363636363633 //// 1_and_1___ 0.2727272727272727 0.45  ====  20
........................................................................................... 
M_Septile_cut_6.0 _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.33531746031746035 //// 1_and_1___ 0.2777777777777778 0.39285714285714285  ====  28
........................... 
_48_cut_up__intFM_category_septil _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3516483516483516 //// 1_and_1___ 0.34615384615384615 0.35714285714285715  ====  42
............................................................................................................................. 
_aux_TRX_CAIDA_30PERCENT_OTROS_NO_TABLERO_CASH_OUT_MES_ANT _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.3252032520325203 //// 0_and_1___ 0.3333333333333333 0.3170731707317073  ====  41
.................................................................................................................................................................................................................. 
TIENE_DEUDA_OTROS_VIGENTE_RCC _0.8_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3392857142857143 //// 1_and_0___ 0.42857142857142855 0.25  ====  8
.......................................................... 
_2_cut_up_BUCKET _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3461210571184996 //// 1_and_1___ 0.2608695652173913 0.43137254901960786  ====  51
................................... 
_2_cut_up_CEF_ROTACION_INVENTARIOS _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.32936507936507936 //// 1_and_1___ 0.21428571428571427 0.4444444444444444  ====  27
............................................................................................................................................................................................................................................................................. 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _0.2_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC 0.4323529411764706 //// 0_and_1___ 0.6 0.2647058823529412  ====  34
......................... 
_0.9_cut_up_CEF_DTOTAL_S_EBITDA_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.356060606060606 //// 1_and_1___ 0.16666666666666666 0.5454545454545454  ====  11
................................................................................ 
_2_cut_up_VARIAICION_NUM_BANCOS_PRIN_DEBT_u12m _0.6_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC 0.35064935064935066 //// 1_and_0___ 0.42857142857142855 0.2727272727272727  ====  11
............ 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M DEBT_CAIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF 0.3977272727272727 //// 0_and_1___ 0.5454545454545454 0.25  ====  36
................................................................................. 
_1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _4_cut_up_M_Septile 0.3333333333333333 //// 1_and_1___ 0.4230769230769231 0.24358974358974358  ====  78
................................................................................................................................................ 
TIENE_DEUDA_OTROS_VIGENTE_RCC TIENE_ADEX_AMERICA_DEL_NORTE_IMP 0.3392857142857143 //// 1_and_0___ 0.42857142857142855 0.25  ====  8
...................................................................................... 
ADEX_CAIDA_20PERCENT_MES_ANT_IMP _int_COVID_ACOGIO_SUSPENSION_PERFECTA 0.3418803418803419 //// 1_and_1___ 0.2222222222222222 0.46153846153846156  ====  13
..................................................................................... 
R_Septile_cut_nan TIENE_ADEX_AGRO_TRADICIONAL_IMP 0.3333333333333333 //// 0_and_1___ 0.6666666666666666 0.0  ====  6
.................................................................................................................................................................................................................................................................................................................................. 
TIENE_ADEX_MADERAS_IMP _0.5_cut_down_CEF_CAPITAL_TRABAJO_VAR 0.3625 //// 1_and_0___ 0.6 0.125  ====  16
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_aux_TRX_CAIDA_20PERCENT_OPERACIONES_MESA__CASH_OUT_MES_ANT _2_cut_up_NUM_VECES_REACTIVA_SF 0.37142857142857144 //// 1_and_1___ 0.6 0.14285714285714285  ====  7
......................................................................................................................................................................................................................................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _1_cut_up_cut_0_TIENE_EL_MISMO_NUM_TIPOS_CAHIN_EN_U3M 0.4650793650793651 //// 0_and_1___ 0.6190476190476191 0.3111111111111111  ====  45
.... 
_aux_TRX_CAIDA_30PERCENT_COBRANZAS__MES_ANT IS_NORMAL_IBK_SBS 0.3944444444444445 //// 0_and_0___ 0.3888888888888889 0.4  ====  80
............................................................................................................................................................................................................................................................................................................................................................................................................. 
TIENE_TRX_TRANSFERENCIAS__CASH_OUT_MES_ACTUAL IS_NORMAL_IBK_SBS 0.41666666666666663 //// 1_and_0___ 0.5 0.3333333333333333  ====  21
. 
TIENE_TRX_TRANSFERENCIAS__CASH_OUT_MES_ACTUAL IS_NORMAL_IBK_SBS 0.37853107344632764 //// 0_and_0___ 0.3333333333333333 0.423728813559322  ====  59
...................................................................................................................... 
_aux_TRX_SUBIDA_20PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT TIENE_DEUDA_OTROS_VIGENTE_RCC 0.3392857142857143 //// 0_and_1___ 0.42857142857142855 0.25  ====  8
.................... 
F_Septile_cut_3.0 IS_NORMAL_IBK_SBS 0.4022366522366523 //// 0_and_0___ 0.3888888888888889 0.4155844155844156  ====  77
................................................................................... 
_aux_TRX_CAIDA_SEGUIDA_TRANSFERENCIAS__U3M_CASH_OUT IS_NORMAL_IBK_SBS 0.3712757830404889 //// 0_and_0___ 0.35294117647058826 0.38961038961038963  ====  77
................................................................................................................................................................................................................. 
TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC _0.999_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.3333333333333333 //// 1_and_0___ 0.3333333333333333 0.3333333333333333  ====  6
.............................................................................................................................................................................................................................. 
_48_cut_up__intFM_category_septil _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.4 //// 1_and_1___ 0.4 0.4  ====  10
..................................................... 
_aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M _7_cut_up_M_Septile 0.46739130434782605 //// 0_and_1___ 0.5652173913043478 0.3695652173913043  ====  46
.............................................................. 
TIENE_ADEX_AGRO_TRADICIONAL_IMP ZONAL__BEL ZONAL 1 0.3333333333333333 //// 1_and_0___ 0.6666666666666666 0.0  ====  6
.......................................................................................................... 
_4_cut_up_R_Quartile cut_letal_SUBSECTOR 0.353125 //// 1_and_1___ 0.3 0.40625  ====  32
.................................................................................................................................................................................................................................................... 
_1.1_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _3_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M 0.5625 //// 1_and_0___ 0.75 0.375  ====  8
................................................................ 
_aux_TRX_CAIDA_SEGUIDA_OTROS_NO_TABLERO_U3M_CASH_OUT TIENE_ADEX_AGRO_TRADICIONAL_IMP 0.3333333333333333 //// 0_and_1___ 0.6666666666666666 0.0  ====  6
..................................................................................... 
cut_02_INGRESO_SUPERA_EGGRESO_SOLO_EN_20PERCENT_O_MENOS _1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3970588235294118 //// 1_and_1___ 0.5 0.29411764705882354  ====  17
....................................... 
_0.9_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _30_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.4311145510835913 //// 1_and_1___ 0.4411764705882353 0.42105263157894735  ====  57
.............................................. 
_aux_TIENE_TRX_OTROS_NO_TABLERO_U3M IS_NORMAL_IBK_SBS 0.4125490196078431 //// 0_and_0___ 0.4117647058823529 0.41333333333333333  ====  75
............................................................................................................................... 
IS_NORMAL_IBK_SBS TIENE_ADEX_OCEANIA_EXP 0.3944444444444445 //// 0_and_0___ 0.3888888888888889 0.4  ====  80
...................................... 
TIENE_SOLO_2_TIPO_CASH_IN_ACTIVO_ESTE_MES _3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.32512733446519526 //// 0_and_1___ 0.2631578947368421 0.3870967741935484  ====  31
................................................................................................................................................................................................................................................................................................................... 
TRX_CAIDA_50PERCENT_MES_ANTERIOR TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC 0.39285714285714285 //// 0_and_1___ 0.5 0.2857142857142857  ====  7
....................................................................... 
cut_3_TIENE_DISMINUCION_3_O_MAS_TIPOS_CASH_IN_U3M TIENE_ADEX_AGRO_TRADICIONAL_IMP 0.3333333333333333 //// 0_and_1___ 0.6666666666666666 0.0  ====  6
................................................................................ 
IS_NORMAL_IBK_SBS cat_TIPOS_CASH_IN_ACTIVOS_U3M__5 a mas 0.3944444444444445 //// 0_and_0___ 0.3888888888888889 0.4  ====  80
...... 
_aux_TIENE_TRX_OTROS_NO_TABLERO_CASH_OUT_U3M _2_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M 0.3399122807017544 //// 0_and_1___ 0.4166666666666667 0.2631578947368421  ====  19
......... 
_aux_TRX_MAYOR_PAGO_DE_SERVICIOS__DE_U3M_CASH_OUT TIENE_ADEX_MADERAS_IMP 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
............................................................................. 
cut_09_EGRESO_SUPERA_INGRESO_EN_10PERCENT_O_MAS _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.36813186813186816 //// 0_and_1___ 0.42857142857142855 0.3076923076923077  ====  39
................................................................................................................. 
_aux_TRX_CAIDA_20PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3333333333333333 //// 1_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
................................... 
_aux_TRX_SUBIDA_20PERCENT_PAGO_DE_SERVICIOS__CASH_OUT_MES_ANT _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.3333333333333333 //// 1_and_1___ 0.5 0.16666666666666666  ====  6
......................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _4_cut_up_M_Septile 0.4508417508417508 //// 0_and_1___ 0.5925925925925926 0.3090909090909091  ====  55
..................................................................................................................................................................... 
_205_cut_up_CEF_ROTACION_CTASXPAGAR _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.39285714285714285 //// 1_and_1___ 0.2857142857142857 0.5  ====  10
......................................................................................................................................................................................................................................... 
_2_cut_up_NUM_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.6785714285714286 //// 0_and_1___ 0.5 0.8571428571428571  ====  7
........................................................................................... 
_0.2_cut_down_SOW_IBK_ULT_RCC _2_cut_up_NUM_BANCOS_CON_MAYOR_SOW_ULT_RCC 0.43062200956937796 //// 0_and_1___ 0.5454545454545454 0.3157894736842105  ====  19
................................................. 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _1_cut_up_NUM_BANCOS_MONTO_DEUDA_MAYOR_QUE_IBKSF_ULT_RCC 0.36934235976789165 //// 1_and_1___ 0.4727272727272727 0.26595744680851063  ====  94
................... 
_1_cut_up_NUM_BANCOS_NOIBK_SUBIDA_30PERCENT_MES_ANT _48_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3431372549019608 //// 1_and_1___ 0.35294117647058826 0.3333333333333333  ====  30
......................................................................................................................................................................................................................... 
IS_NORMAL_IBK_SBS _4_cut_up_F_Septile 0.38281979458450044 //// 0_and_1___ 0.35294117647058826 0.4126984126984127  ====  63
 
IS_NORMAL_IBK_SBS _4_cut_up_F_Septile 0.6764705882352942 //// 0_and_0___ 1.0 0.35294117647058826  ====  17
........................................................................................................................................................................................................................................................................................ 
_3_cut_up_NUM_BANCOS_CON_MAYOR_SOW_ULT_RCC cut_letal_SUBSECTOR 0.5625 //// 1_and_1___ 0.5 0.625  ====  8
....................................................................... 
NRO_BANCOS_NOIBK_SUBIDA_10PERCENT_MES_ANT _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.4017857142857143 //// 1_and_1___ 0.375 0.42857142857142855  ====  14
................................................... 
TIENE_OTRA_ENTIDAD_TARJETACREDIT_Y_COMEX_VIGENTES_SF_ULT_RCC _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.32741935483870965 //// 1_and_1___ 0.3 0.3548387096774194  ====  31
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
_0.5_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.34330143540669855 //// 0_and_1___ 0.3181818181818182 0.3684210526315789  ====  57
....... 
_5_cut_up_CEF_DTOTAL_S_EBITDA_VAR _3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.35416666666666663 //// 0_and_1___ 0.3333333333333333 0.375  ====  32
................................................................. 
F_Septile_cut_1.0 _1_cut_up_VARIAICION_NUM_BANCOS_CIERRE_2019 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  9
............... 
M_Septile_cut_6.0 _2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.37662337662337664 //// 1_and_1___ 0.5714285714285714 0.18181818181818182  ====  11
.................................................. 
IS_NORMAL_IBK_SBS TIENE_ADEX_AMERICA_CENTRAL_EXP 0.3944444444444445 //// 0_and_0___ 0.3888888888888889 0.4  ====  80
 
DEBT_TOTAL_CAIDA_10PERCENT_MES_ANT _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.5833333333333333 //// 1_and_1___ 0.5 0.6666666666666666  ====  6
........................................................... 
NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.6818181818181819 //// 1_and_1___ 1.0 0.36363636363636365  ====  11
..................................................................................................................................... 
cut_1_TIENE_AL_MENOS_1_TIPO_CASHIN_ACTIVO_U3M COVID_ACOGIO_SUSPENSION_PERFECTA_SI 0.34285714285714286 //// 0_and_1___ 0.4 0.2857142857142857  ====  7
....................... 
TIENE_TRX_OPERACIONES_MESA__CASH_OUT_MES_ACTUAL TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
....................................... 
_aux_TRX_CAIDA_10PERCENT_OPERACIONES_MESA__CASH_OUT_MES_ANT TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC 0.3392857142857143 //// 0_and_1___ 0.42857142857142855 0.25  ====  8
..................................................................................................................................................................................................................... 
TRX_SUBIDA_20PERCENT_MES_ANTERIOR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38125 //// 1_and_1___ 0.2 0.5625  ====  16
........... 
TIENE_DISMINUCION_1_O_MAS_TIPOS_CASH_OUT_U3M TIENE_DEUDA_OTROS_VIGENTE_RCC 0.3666666666666667 //// 1_and_1___ 0.4 0.3333333333333333  ====  6
................................................................................................................................................................................................. 
_0.2_cut_down_SOW_IBK_ULT_RCC _2_cut_up_NUM_BANCOS_CON_MAYOR_SOW_ULT_RCC 0.43062200956937796 //// 0_and_1___ 0.5454545454545454 0.3157894736842105  ====  19
............................................................................. 
TICKET_PROMEDIO_EN_RIESGO__Bucket 2 - 1.0 MM _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.5256410256410257 //// 1_and_1___ 0.6666666666666666 0.38461538461538464  ====  13
............................................. 
_0.05_cut_down_COVER_GAR_AUTO_ULT_RCC _20_cut_up_COVID_PORCENTAJE_REDUCCION_INGREO_MENUSAL 0.3333333333333333 //// 0_and_1___ 0.3333333333333333 0.3333333333333333  ====  6
..................................................................................................... 
_3_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M _3_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.5 //// 1_and_1___ 0.5 0.5  ====  10
................................................................................................................................................................. 
NO_TIENE_TRX_PAGOS_RECIBIDOS__MES_ACTUAL _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.45 //// 0_and_1___ 0.4 0.5  ====  6
........................................................................................................................................................................................................................................................................................................................... 
_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.3894736842105263 //// 0_and_1___ 0.2 0.5789473684210527  ====  19
....................................... 
_aux_TRX_CAIDA_20PERCENT_DEPOSITOS__MES_ANT TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC 0.3666666666666667 //// 0_and_1___ 0.4 0.3333333333333333  ====  6
........................................................................................................................................................................................... 
IS_NORMAL_IBK_SBS _4_cut_up_R_Quartile 0.4519230769230769 //// 0_and_1___ 0.4166666666666667 0.48717948717948717  ====  39
 
IS_NORMAL_IBK_SBS _4_cut_up_R_Quartile 0.3252032520325203 //// 0_and_0___ 0.3333333333333333 0.3170731707317073  ====  41
............................................................................................................................................................................................................................. 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC 0.343680709534368 //// 1_and_0___ 0.4146341463414634 0.2727272727272727  ====  66
.................................................. 
_1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.45 //// 1_and_1___ 0.4 0.5  ====  8
............................................... 
_1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _0.005_cut_down_PERCENT_GAR_HIPO 0.34134615384615385 //// 1_and_1___ 0.375 0.3076923076923077  ====  13
..................................................................................................................................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M BACKLOG__No Aplica 0.4095238095238095 //// 0_and_1___ 0.5333333333333333 0.2857142857142857  ====  42
....................................................................................................................................................... 
TIENE_SOLO_3_TIPO_CASH_IN_ACTIVO_ESTE_MES _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP12_PERCENT 0.34330143540669855 //// 0_and_1___ 0.3181818181818182 0.3684210526315789  ====  38
........... 
_0.4_cut_down_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.3306808134394341 //// 0_and_1___ 0.3793103448275862 0.28205128205128205  ====  39
................................. 
F_Septile_cut_4.0 _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.6666666666666666 //// 1_and_1___ 1.0 0.3333333333333333  ====  9
................................................................... 
_36_cut_up__intFM_category_septil _12_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.3529411764705882 //// 1_and_1___ 0.29411764705882354 0.4117647058823529  ====  34
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... 
M_Septile_cut_6.0 _2_cut_up_CEF_DTOTAL_S_EBITDA_VAR 0.3455265610438024 //// 1_and_1___ 0.43243243243243246 0.25862068965517243  ====  58
............................................................................................................................................................... 
_2_cut_up_NUM_VECES_REACTIVA_IBK _50_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38888888888888884 //// 1_and_1___ 0.3333333333333333 0.4444444444444444  ====  9
......................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC 0.4030769230769231 //// 0_and_1___ 0.56 0.24615384615384617  ====  65
................................................................................................. 
FLG_SUSPENDIDO _4_cut_up_R_Quartile 0.3415233415233415 //// 1_and_1___ 0.3181818181818182 0.36486486486486486  ====  74
................................................................................................................................................................... 
_3_cut_up_BUCKET _3_cut_up_NUM_VECES_REACTIVA_SF 0.35416666666666663 //// 1_and_1___ 0.3333333333333333 0.375  ====  8
..... 
_aux_TRX_SUBIDA_20PERCENT_PAGO_DE_SERVICIOS__CASH_OUT_MES_ANT TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC 0.3392857142857143 //// 0_and_1___ 0.42857142857142855 0.25  ====  8
................................. 
TIENE_ADEX_MADERAS_IMP _2_cut_up_CEF_DTOTAL_S_EBITDA_VAR 0.55 //// 1_and_1___ 1.0 0.1  ====  10
....................... 
NRO_BANCOS_DEBT_SUBIDA_SEGUIDA_DE_U3M COVID_ACOGIO_SUSPENSION_PERFECTA_SI 0.3351648351648352 //// 1_and_1___ 0.2857142857142857 0.38461538461538464  ====  13
................................................................................................... 
TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_SF_ULT_RCC COVID_ACOGIO_SUSPENSION_PERFECTA_SI 0.34285714285714286 //// 1_and_1___ 0.2857142857142857 0.4  ====  15
....................................................................................................................................... 
TIENE_SOLO_1_TIPO_CASH_IN_ACTIVO_ESTE_MES _1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.41785714285714287 //// 1_and_1___ 0.55 0.2857142857142857  ====  35
....................................................... 
_200_cut_up_CEF_ROTACION_INVENTARIOS _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.33585858585858586 //// 1_and_1___ 0.22727272727272727 0.4444444444444444  ====  45
............................................................................................................................................................................................................................................................................................................................................................................ 
TIENE_DEUDA_OTROS_VIGENTE_RCC _1_cut_up_NUM_BANCOS_DEBT_SUBIDA_SEGUIDA_DE_U3M 0.3571428571428571 //// 1_and_0___ 0.42857142857142855 0.2857142857142857  ====  7
.................................................................................................... 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC 0.4030769230769231 //// 0_and_1___ 0.56 0.24615384615384617  ====  65
............................................................... 
cat_CASHIN_ACTIVOS_MES_ACTUAL__3 a 4 _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.3392857142857143 //// 0_and_1___ 0.35714285714285715 0.32142857142857145  ====  28
.............................................................................................................................................................. 
TIENE_OTROS_Y_COMEX_VIGENTES_SF_ULT_RCC TIENE_ADEX_MADERAS_EXP 0.3392857142857143 //// 1_and_0___ 0.42857142857142855 0.25  ====  8
.................................................. 
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _420_cut_up_CEF_ROTACION_INVENTARIOS 0.34285714285714286 //// 1_and_1___ 0.4 0.2857142857142857  ====  77
....... 
TIENE_DEUDA_PRESTAMOS_VIGENTE_RCC TIENE_ADEX_AGRO_TRADICIONAL_IMP 0.3333333333333333 //// 1_and_1___ 0.6666666666666666 0.0  ====  6
................................................................................................................................................................................... 
DEBT_TOTAL_SUBIDA_30PERCENT_MES_ANT _1_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4375 //// 1_and_1___ 0.625 0.25  ====  12
................................................... 
TRX_CAIDA_40PERCENT_TOTAL_CASH_OUT_MES_ANT _int_COVID_ACOGIO_SUSPENSION_PERFECTA_Y_REDUCCION_PEROSNAL_UP20_PERCENT 0.36160714285714285 //// 1_and_1___ 0.2857142857142857 0.4375  ====  16
....................................... 
_0.9_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR _6_cut_up_R_Septile 0.390625 //// 1_and_1___ 0.53125 0.25  ====  48
................................................................. 
_aux_TIENE_TRX_TOTAL_CASH_OUT_U3M _aux_TRX_MAYOR_TOTAL_DE_U3M_CASH_OUT 0.5606060606060606 //// 0_and_1___ 0.6666666666666666 0.45454545454545453  ====  11
..................................................................................................................................................................................................................... 
R_Septile_cut_7.0 cut_letal_SUBSECTOR 0.4 //// 1_and_1___ 0.3 0.5  ====  16
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................. 
IS_NORMAL_IBK_SBS _2_cut_up_BUCKET 0.5833333333333333 //// 0_and_1___ 0.5 0.6666666666666666  ====  18
 
IS_NORMAL_IBK_SBS _2_cut_up_BUCKET 0.33986175115207373 //// 0_and_0___ 0.35714285714285715 0.3225806451612903  ====  62

In [ ]:
# SELCCIONADAS MEDIANA

_3_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  8
TIENE_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38888888888888884 //// 0.3333333333333333 0.4444444444444444  ====  9
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _48_cut_up__intFM_category_septil 0.4772727272727273 //// 0.5 0.45454545454545453  ====  11

cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M DEBT_SUBIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.5 //// 1.0 0.0  ====  9

TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  16
_2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _3_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.5 //// 1.0 0.0  ====  6
DEBT_SUBIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4625 //// 0.625 0.3  ====  20
TRX_SUBIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT _7_cut_up_M_Septile 0.41666666666666663 //// 0.5 0.3333333333333333  ====  6
TIENE_OTRA_ENTIDAD_TARJETACREDIT_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 0.3333333333333333 0.5  ====  8
cat_CASHIN_ACTIVOS_MES_ACTUAL__hasta 2 _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38095238095238093 //// 0.3333333333333333 0.42857142857142855  ====  7
DEBT_MAYOR_MTOSALDO_IBK_DE_U3M_SF _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.42361111111111116 //// 0.5555555555555556 0.2916666666666667  ====  24
BACKLOG__No Aplica _2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.6666666666666666 //// 1.0 0.3333333333333333  ====  6
TIENE_ADEX_MINERIA_NO_METALICA_EXP _1_cut_up_CEF_DTOTAL_S_VENTAS_VAR 0.5 //// 1.0 0.0  ====  7
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _36_cut_up__intFM_category_septil 0.40350877192982454 //// 0.3333333333333333 0.47368421052631576  ====  19
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR cut_deCuidado_SUBSECTOR 0.3666666666666667 //// 0.4 0.3333333333333333  ====  12
NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC _36_cut_up__intFM_category_septil 0.4358974358974359 //// 0.3333333333333333 0.5384615384615384  ====  13
_1_cut_up_CANTIDAD_TRX_CASH_OUT_PERCENT _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.4375 //// 0.5 0.375  ====  8
_0.25_cut_down_SOW_IBK_MAX_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6111111111111112 //// 1.0 0.2222222222222222  ====  9
_0.2_cut_down_SOW_IBK_MIN_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6 //// 1.0 0.2  ====  10

In [ ]:
_3_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  8
TIENE_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38888888888888884 //// 0.3333333333333333 0.4444444444444444  ====  9
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _48_cut_up__intFM_category_septil 0.4772727272727273 //// 0.5 0.45454545454545453  ====  11
cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M DEBT_SUBIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.5 //// 1.0 0.0  ====  9
TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  16
_2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _3_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.5 //// 1.0 0.0  ====  6
DEBT_SUBIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4625 //// 0.625 0.3  ====  20
TRX_SUBIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT _7_cut_up_M_Septile 0.41666666666666663 //// 0.5 0.3333333333333333  ====  6
TIENE_OTRA_ENTIDAD_TARJETACREDIT_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 0.3333333333333333 0.5  ====  8
cat_CASHIN_ACTIVOS_MES_ACTUAL__hasta 2 _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38095238095238093 //// 0.3333333333333333 0.42857142857142855  ====  7
DEBT_MAYOR_MTOSALDO_IBK_DE_U3M_SF _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.42361111111111116 //// 0.5555555555555556 0.2916666666666667  ====  24
BACKLOG__No Aplica _2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.6666666666666666 //// 1.0 0.3333333333333333  ====  6
TIENE_ADEX_MINERIA_NO_METALICA_EXP _1_cut_up_CEF_DTOTAL_S_VENTAS_VAR 0.5 //// 1.0 0.0  ====  7
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _36_cut_up__intFM_category_septil 0.40350877192982454 //// 0.3333333333333333 0.47368421052631576  ====  19
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR cut_deCuidado_SUBSECTOR 0.3666666666666667 //// 0.4 0.3333333333333333  ====  12
NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC _36_cut_up__intFM_category_septil 0.4358974358974359 //// 0.3333333333333333 0.5384615384615384  ====  13
_1_cut_up_CANTIDAD_TRX_CASH_OUT_PERCENT _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.4375 //// 0.5 0.375  ====  8
_0.25_cut_down_SOW_IBK_MAX_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6111111111111112 //// 1.0 0.2222222222222222  ====  9
_0.2_cut_down_SOW_IBK_MIN_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6 //// 1.0 0.2  ====  10

In [ ]:
_3_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M _20_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  8
TIENE_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38888888888888884 //// 0.3333333333333333 0.4444444444444444  ====  9
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _48_cut_up__intFM_category_septil 0.4772727272727273 //// 0.5 0.45454545454545453  ====  11
cut_2_TIENE_DISMINUCION_2_O_MAS_TIPOS_CASH_IN_U3M DEBT_SUBIDA_10PERCENT_MTOSALDO_IBK_SF_U3M 0.5 //// 1.0 0.0  ====  9
TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.375 //// 0.5 0.25  ====  16
_2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR _3_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M 0.5 //// 1.0 0.0  ====  6
DEBT_SUBIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF _1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.4625 //// 0.625 0.3  ====  20
TRX_SUBIDA_30PERCENT_TOTAL_CASH_OUT_MES_ANT _7_cut_up_M_Septile 0.41666666666666663 //// 0.5 0.3333333333333333  ====  6
TIENE_OTRA_ENTIDAD_TARJETACREDIT_Y_COMEX_VIGENTES_SF_ULT_RCC _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.41666666666666663 //// 0.3333333333333333 0.5  ====  8
cat_CASHIN_ACTIVOS_MES_ACTUAL__hasta 2 _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.38095238095238093 //// 0.3333333333333333 0.42857142857142855  ====  7
DEBT_MAYOR_MTOSALDO_IBK_DE_U3M_SF _1.5_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR 0.42361111111111116 //// 0.5555555555555556 0.2916666666666667  ====  24
BACKLOG__No Aplica _2_cut_up_CEF_ROTACION_CTASXCOBRAR_VAR 0.6666666666666666 //// 1.0 0.3333333333333333  ====  6
TIENE_ADEX_MINERIA_NO_METALICA_EXP _1_cut_up_CEF_DTOTAL_S_VENTAS_VAR 0.5 //// 1.0 0.0  ====  7
NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _36_cut_up__intFM_category_septil 0.40350877192982454 //// 0.3333333333333333 0.47368421052631576  ====  19
_1.2_cut_up_CEF_ENDEUDAMIENTO_PAS_VEN_VAR cut_deCuidado_SUBSECTOR 0.3666666666666667 //// 0.4 0.3333333333333333  ====  12
NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC _36_cut_up__intFM_category_septil 0.4358974358974359 //// 0.3333333333333333 0.5384615384615384  ====  13
_1_cut_up_CANTIDAD_TRX_CASH_OUT_PERCENT _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC 0.4375 //// 0.5 0.375  ====  8
_0.25_cut_down_SOW_IBK_MAX_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6111111111111112 //// 1.0 0.2222222222222222  ====  9
_0.2_cut_down_SOW_IBK_MIN_historia _30_cut_up_COVID_PORCENTAJE_REDUCCION_PERSONAL 0.6 //// 1.0 0.2  ====  10